<a href="https://colab.research.google.com/github/sercantsn6-droid/sercantsn6-droid/blob/main/KodunSonHali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio  #Deneme1
# Birleşik LeNet CNN Katman Hızlandırıcı Arayüzü
import tensorflow as tf
import numpy as np
import gradio as gr
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from google.colab import drive

drive.mount('/content/drive')

# Model ve veri yolları
MODELS = {
    "LeNet1": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
        "flow_from_directory": True
    },
    "LeNet2": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
        "flow_from_directory": False
    },
    "LeNet3": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
        "flow_from_directory": False
    }
}

# Global değişkenler
test_data_cache = {}
model_cache = {}
layer_cache = {}

# Yardımcı fonksiyonlar
def load_model_and_data(model_name):
    config = MODELS[model_name]
    model = tf.keras.models.load_model(config["model_path"])

    if config["flow_from_directory"]:
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
        test_data = datagen.flow_from_directory(
            config["test_path"], target_size=(32, 32), batch_size=8,
            class_mode=None, shuffle=False)
    else:
        image_files = [os.path.join(config["test_path"], f) for f in os.listdir(config["test_path"]) if f.endswith(('.png', '.jpg', '.jpeg'))]
        images = []
        for img_path in image_files:
            img = load_img(img_path, target_size=(32, 32))
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
        test_data = tf.data.Dataset.from_tensor_slices(np.array(images)).batch(8)

    layer_names = [f"Katman {i}: {layer.name} - Çıkış Şekli: {layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'}" for i, layer in enumerate(model.layers)]

    model_cache[model_name] = model
    test_data_cache[model_name] = test_data
    layer_cache[model_name] = layer_names

    return layer_names

def get_layer_output(model, layer_index, input_data):
    intermediate_model = tf.keras.Sequential(model.layers[:layer_index+1])
    return intermediate_model.predict(input_data, verbose=0)

def run_layer(model_name, layer_selection, data_choice):
    model = model_cache[model_name]
    test_data = test_data_cache[model_name]

    layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
    current_layer = model.layers[layer_index]

    if data_choice == "Test Verisinden Örnek Kullan":
        input_data = next(iter(test_data))
        data_info = f"\u00d6rnek veri kullan\u0131ld\u0131. \u015eekil: {input_data.shape}"
    else:
        expected_shape = model.input.shape[1:]
        input_data = np.random.random((8, *expected_shape))
        data_info = f"Rastgele veri olu\u015fturuldu. \u015eekil: {input_data.shape}"

    try:
        real_output = get_layer_output(model, layer_index, input_data)
        fpga_output = real_output + np.random.normal(0, 0.05, real_output.shape)

        real_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized = real_np.flatten().tobytes()

        mse = np.mean((real_np - fpga_np)**2)
        similarity = max(0, 1 - mse)

        expected_input = model.input.shape[1:] if layer_index == 0 else model.layers[layer_index - 1].output.shape[1:]

        output_str = f"Se\u00e7ilen Katman: {current_layer.name}\n"
        output_str += f"Beklenen Giri\u015f \u015eekli: {expected_input}\n"
        output_str += f"Ger\u00e7ek Giri\u015f \u015eekli: {input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Ger\u00e7ek \u00c7\u0131kt\u0131 \u015eekli: {real_output.shape}\n"
        output_str += f"FPGA \u00c7\u0131kt\u0131 \u015eekli: {fpga_output.shape}\n"
        output_str += f"Donan\u0131ma G\u00f6nderilen \u00c7\u0131kt\u0131 Boyutu: {len(serialized)} bayt\n"
        output_str += f"Ger\u00e7ek \u00c7\u0131kt\u0131 \u00d6rnek Veri: {real_np.flatten()[:5]}\n"
        output_str += f"FPGA \u00c7\u0131kt\u0131 \u00d6rnek Veri: {fpga_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1\u2019e yak\u0131nsa daha uyumlu)\n"

        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA \u00c7\u0131kt\u0131s\u0131n\u0131n Ger\u00e7ek \u00c7\u0131kt\u0131yla Uyumu</p>
        """
        return output_str, animation_html

    except Exception as e:
        return f"Hata: {str(e)}", "<p>Hata nedeniyle animasyon olu\u015fturulamad\u0131.</p>"

# Gradio aray\u00fcz\u00fc
with gr.Blocks(title="Birle\u015fik LeNet CNN Katman H\u0131zland\u0131r\u0131c\u0131") as interface:
    gr.Markdown("# Birle\u015fik LeNet CNN Katman H\u0131zland\u0131r\u0131c\u0131")

    model_dropdown = gr.Dropdown(choices=list(MODELS.keys()), label="Model Se\u00e7in")
    layer_dropdown = gr.Dropdown(choices=[], label="Katman Se\u00e7in")
    data_choice = gr.Radio(choices=["Test Verisinden \u00d6rnek Kullan", "Rastgele Veri Kullan"], label="Giri\u015f Verisi Se\u00e7imi")
    run_button = gr.Button("Katman\u0131 \u00c7al\u0131\u015ft\u0131r")
    result_text = gr.Textbox(label="Sonu\u00e7lar", lines=15)
    result_anim = gr.HTML()

    def update_layers(model_name):
        layer_names = load_model_and_data(model_name)
        return gr.Dropdown.update(choices=layer_names, value=layer_names[0])

    model_dropdown.change(update_layers, inputs=model_dropdown, outputs=layer_dropdown)
    run_button.click(fn=run_layer, inputs=[model_dropdown, layer_dropdown, data_choice], outputs=[result_text, result_anim])

interface.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00
Mounted at /content/drive
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://caccc51c06c497c81a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces 

In [ ]:
!pip install gradio        #TASLAK1 WITH SIMULATION
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from google.colab import drive
import gradio as gr
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Connect Google Drive
drive.mount('/content/drive')

# Define model and data paths
MODELS = {
    "LeNet1": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
        "flow_from_directory": True
    },
    "LeNet2": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
        "flow_from_directory": False
    },
    "LeNet3": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
        "flow_from_directory": False
    }
}

# Data loading functions
# For directories without subfolders
def load_images_from_directory(directory, target_size=(32, 32), batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0  # Normalization
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

# For directories with subfolders
def load_images_with_flow_from_directory(directory, target_size=(32, 32), batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Layer output function
def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# FPGA simulation and comparison
def run_layer(model_selection, layer_selection, data_choice):
    # Get model and data paths
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Load model
    loaded_model = tf.keras.models.load_model(model_path)

    # Load test data
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(32, 32), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(32, 32), batch_size=8)

    # Get the selected layer index
    layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
    current_layer = loaded_model.layers[layer_index]

    # Data selection and input preparation
    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)  # Take the batch from ImageDataGenerator
        else:
            input_data = next(iter(test_data))  # Take the batch from tf.data.Dataset
        data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]  # (32, 32, 3)
        input_data = np.random.random((8, *expected_shape))  # compatible with batch size 8
        data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"

    try:
        # Run all layers up to the selected layer
        real_output = get_layer_output(loaded_model, layer_index, input_data)

        # Simulated output from FPGA
        fpga_output = real_output + np.random.normal(0, 0.05, real_output.shape)

        # Convert output to NumPy array
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        # Similarity calculation (with MSE)
        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)  # 0-1 arasında skor

        # Expected input shape
        expected_input_shape = loaded_model.input.shape[1:] if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape[1:]

        # Prepare the results
        output_str = f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Donanıma Gönderilen Çıktı Boyutu: {len(serialized_output)} bayt\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        # Animated HTML output
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Function to dynamically retrieve layer information
def update_layer_dropdown(model_selection):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    loaded_model = tf.keras.models.load_model(model_path)
    layer_names = [f"Katman {i}: {layer.name} - Çıkış Şekli: {layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'}"
                   for i, layer in enumerate(loaded_model.layers)]
    return gr.Dropdown(choices=layer_names, value=layer_names[0])

# Create the Gradio interface
with gr.Blocks(title="LeNet CNN Katman Hızlandırıcı") as interface:
    gr.Markdown("# LeNet CNN Katman Hızlandırıcı")
    gr.Markdown("Modelinizi seçin, ardından katmanınızı ve giriş verisi tipini belirleyin. FPGA çıktısının gerçek çıktı ile uyumunu görün!")

    # Model selection
    with gr.Row():
        model_input = gr.Dropdown(choices=list(MODELS.keys()), label="Model Seçimi", value="LeNet1")

    # Layer selection and simülation
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin")
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Katmanı Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Update layer dropdown when model is selected
    model_input.change(fn=update_layer_dropdown, inputs=model_input, outputs=layer_input)

    # Run the layer
    submit_btn.click(fn=run_layer, inputs=[model_input, layer_input, data_input], outputs=[output_text, output_animation])

# Start the interface
interface.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.7 MB/s eta 0:00:00
Mounted at /content/drive
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0db4473952dd5614ac.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal

In [ ]:
!pip install gradio                                   #TASLAK1 WITHOUT SIMULATION  #Last Model
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from google.colab import drive
import gradio as gr
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import io

# Mount Google Drive
drive.mount('/content/drive')

# Define model and data paths
MODELS = {
    "LeNet1": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
        "flow_from_directory": True
    },
    "LeNet2": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
        "flow_from_directory": False
    },
    "LeNet3": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
        "flow_from_directory": False
    }
}

# Data loading functions
# For directories without subfolders

def load_images_from_directory(directory, target_size=(32, 32), batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"Directory {directory} not found!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0  # Normalize
            images.append(img_array)
    if not images:
        raise ValueError(f"No valid image files found in directory {directory}!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

# For directories with subfolders

def load_images_with_flow_from_directory(directory, target_size=(32, 32), batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Function to export layer input

def export_layer_input(model_selection, layer_selection, data_choice):
    # Get model and data paths
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Load the model
    loaded_model = tf.keras.models.load_model(model_path)

    # Load test data
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(32, 32), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(32, 32), batch_size=8)

    # Get selected layer index
    layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))

    # Prepare input data based on choice

    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)  # Get batch from ImageDataGenerator
        else:
            input_data = next(iter(test_data))  # Get batch from tf.data.Dataset
        data_info = f"Sample data used. Shape: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]  # (32, 32, 3)
        input_data = np.random.random((8, *expected_shape))  # Compatible with batch size 8
        data_info = f"Random data generated. Shape: {input_data.shape}"

    # If not the first layer, use the output of the previous layer as input
    if layer_index > 0:
        intermediate_model = tf.keras.Model(inputs=loaded_model.input, outputs=loaded_model.layers[layer_index - 1].output)
        input_data = intermediate_model.predict(input_data, verbose=0)

    # Convert input data to hexadecimal format and save
    hex_filename = f"layer_{layer_index}_input_hex.txt"
    flat_data = input_data.flatten().astype(np.float32)  # Flatten and convert to float32
    # Convert data to bytes and format each byte as "0xaa"
    hex_data = []
    for value in flat_data:
        bytes_value = np.float32(value).tobytes()  # Convert float32 to bytes
        hex_bytes = [f"0x{byte:02x}" for byte in bytes_value]  # Format each byte as 0xaa
        hex_data.extend(hex_bytes)

    # Write hex data to file in comma-separated format
    with open(hex_filename, 'w') as f:
        f.write(','.join(hex_data))  # Write as 0xaa,0xbb,...

    # Prepare input information as text

    input_text = f"Layer Input Data Shape: {input_data.shape}\n"
    input_text += f"Sample Data (Float): {flat_data[:5]}\n"
    input_text += f"Sample Data (Hex, first 5 bytes): {hex_data[:5]}\n"
    input_text += f"Input data saved in hexadecimal format to '{hex_filename}'. Download this file to send to Kağan."

    return input_text, gr.File(value=hex_filename)

# Function to get layer output

def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# FPGA simulation and comparison

def run_layer(model_selection, layer_selection, data_choice, fpga_output_file):
    # Get model and data paths
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Load the model
    loaded_model = tf.keras.models.load_model(model_path)

    # Load test data
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(32, 32), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(32, 32), batch_size=8)

    # Get selected layer index
    layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
    current_layer = loaded_model.layers[layer_index]

    # Prepare input data based on choice

    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)  # Get batch from ImageDataGenerator
        else:
            input_data = next(iter(test_data))  # Get batch from tf.data.Dataset
        data_info = f"Sample data used. Shape: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]  # (32, 32, 3)
        input_data = np.random.random((8, *expected_shape))  # Compatible with batch size 8
        data_info = f"Random data generated. Shape: {input_data.shape}"

    try:
        # Run all layers up to the selected layer
        real_output = get_layer_output(loaded_model, layer_index, input_data)

        # Load FPGA output (hex .txt format)
        if fpga_output_file is None:
            return "Please upload the FPGA output!", "<p>FPGA output not uploaded.</p>"

        # Read hex file
        with open(fpga_output_file.name, 'r') as f:
            hex_data = f.read().strip().split(',')

        # Convert hex bytes to float32
        if len(hex_data) % 4 != 0:
            return "Invalid FPGA output! 4 bytes expected per float32.", "<p>Invalid FPGA output.</p>"

        byte_data = [int(h.replace('0x', ''), 16) for h in hex_data]
        float_data = []
        for i in range(0, len(byte_data), 4):
            bytes_value = bytes(byte_data[i:i+4])
            float_value = np.frombuffer(bytes_value, dtype=np.float32)[0]
            float_data.append(float_value)

        # Reshape FPGA output to match real output shape
        fpga_output_np = np.array(float_data).reshape(real_output.shape)

        # Convert outputs to NumPy arrays
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        # Calculate similarity (MSE)
        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)  # Score between 0 and 1

        # Expected input shape
        expected_input_shape = loaded_model.input.shape[1:] if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape[1:]

        # Prepare results
        output_str = f"Selected Model: {model_selection}\n"
        output_str += f"Selected Layer: {current_layer.name}\n"
        output_str += f"Expected Input Shape: {expected_input_shape}\n"
        output_str += f"Actual Input Shape: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Real Output Shape: {real_output.shape}\n"
        output_str += f"FPGA Output Shape: {fpga_output_np.shape}\n"
        output_str += f"Output Size Sent to Hardware: {len(serialized_output)} bytes\n"
        output_str += f"Real Output Sample Data: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Output Sample Data: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Similarity Score: {similarity:.4f} (closer to 1 is more compatible)\n"

        # Animated HTML output
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Output Compatibility with Real Output</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Error occurred: {str(e)}\n"
        error_msg += f"Selected Layer: {current_layer.name}\n"
        error_msg += f"Input Data Shape: {input_data.shape}\n"
        error_msg += f"Valid Layer Output Shape: {real_output.shape if 'real_output' in locals() else 'Not computed'}\n"
        return error_msg, "<p>Animation could not be created due to error.</p>"

# Function to dynamically update layer dropdown

def update_layer_dropdown(model_selection):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    loaded_model = tf.keras.models.load_model(model_path)
    layer_names = [f"Katman {i}: {layer.name} - Output Shape: {layer.output.shape if hasattr(layer, 'output') else 'Input Layer'}"
                   for i, layer in enumerate(loaded_model.layers)]
    return gr.Dropdown(choices=layer_names, value=layer_names[0])

# Create Gradio interface
with gr.Blocks(title="LeNet CNN Layer Accelerator") as interface:
    gr.Markdown("# LeNet CNN Layer Accelerator")
    gr.Markdown("Select your model, then choose the layer and input data type. Export the layer input and upload the FPGA output to compare!")

    # Model selection
    with gr.Row():
        model_input = gr.Dropdown(choices=list(MODELS.keys()), label="Model Selection", value="LeNet1")

    # Layer selection and simulation
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Select Layer to Accelerate")
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Input Data Selection", value="Test Verisinden Örnek Kullan")

            # Export input
            export_btn = gr.Button("Export Layer Input", variant="secondary")
            input_text = gr.Textbox(label="Layer Input Information", lines=3)
            input_file = gr.File(label="Hex File (Download to Send to Kağan)")

            # Upload FPGA output
            fpga_output_file = gr.File(label="Upload FPGA Output from Kağan (hex .txt format)")

            # Comparison button
            submit_btn = gr.Button("Perform Comparison", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Results", lines=15)
            output_animation = gr.HTML(label="FPGA Output Compatibility")

    # Update layer dropdown when model is selected
    model_input.change(fn=update_layer_dropdown, inputs=model_input, outputs=layer_input)

    # Export input
    export_btn.click(fn=export_layer_input, inputs=[model_input, layer_input, data_input], outputs=[input_text, input_file])

    # Perform comparison
    submit_btn.click(fn=run_layer, inputs=[model_input, layer_input, data_input, fpga_output_file], outputs=[output_text, output_animation])

# Launch the interface
interface.launch()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f268632e5e7df51dca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#do not forget to use !pip install gradio we already use it upstairs + üsteki kod son aşama şu anlık burdan itibaren gerçek simülasyon için...
import tensorflow as tf                                #NPY ŞEKLİNDE ÇIKTI
import numpy as np
from tensorflow.keras import layers
from google.colab import drive
import gradio as gr
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import io

# Google Drive'ı bağla
drive.mount('/content/drive')

# Model ve veri yollarını tanımla
MODELS = {
    "LeNet1": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
        "flow_from_directory": True
    },
    "LeNet2": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
        "flow_from_directory": False
    },
    "LeNet3": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
        "flow_from_directory": False
    }
}

# Veri yükleme fonksiyonları
# Alt klasörsüz dizinler için
def load_images_from_directory(directory, target_size=(32, 32), batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0  # Normalizasyon
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

# Alt klasörlü dizinler için
def load_images_with_flow_from_directory(directory, target_size=(32, 32), batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Katman girişini dışa aktarma fonksiyonu
def export_layer_input(model_selection, layer_selection, data_choice):
    # Model ve veri yollarını al
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Modeli yükle
    loaded_model = tf.keras.models.load_model(model_path)

    # Test verisini yükle
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(32, 32), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(32, 32), batch_size=8)

    # Seçilen katman indeksini al
    layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))

    # Veri seçimi ve giriş hazırlanması
    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)  # ImageDataGenerator'dan batch alma
        else:
            input_data = next(iter(test_data))  # tf.data.Dataset'ten batch alma
        data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]  # (32, 32, 3)
        input_data = np.random.random((8, *expected_shape))  # Batch boyutu 8 ile uyumlu
        data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"

    # Eğer ilk katman değilse, önceki katmanların çıktısını giriş olarak al
    if layer_index > 0:
        intermediate_model = tf.keras.Model(inputs=loaded_model.input, outputs=loaded_model.layers[layer_index - 1].output)
        input_data = intermediate_model.predict(input_data, verbose=0)

    # Giriş verisini bir .npy dosyası olarak kaydet
    input_filename = f"layer_{layer_index}_input.npy"
    np.save(input_filename, input_data)

    # Giriş verisini metin olarak da göster
    input_text = f"Katman Giriş Verisi Şekli: {input_data.shape}\n"
    input_text += f"Örnek Veri: {input_data.flatten()[:5]}\n"
    input_text += f"Giriş verisi '{input_filename}' dosyasına kaydedildi. Bu dosyayı indirip Kağan'a gönderebilirsiniz."

    return input_text, gr.File(value=input_filename)

# Katman çıktısını alma fonksiyonu
def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# FPGA simülasyonu ve karşılaştırma
def run_layer(model_selection, layer_selection, data_choice, fpga_output_file):
    # Model ve veri yollarını al
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Modeli yükle
    loaded_model = tf.keras.models.load_model(model_path)

    # Test verisini yükle
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(32, 32), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(32, 32), batch_size=8)

    # Seçilen katman indeksini al
    layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
    current_layer = loaded_model.layers[layer_index]

    # Veri seçimi ve giriş hazırlanması
    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)  # ImageDataGenerator'dan batch alma
        else:
            input_data = next(iter(test_data))  # tf.data.Dataset'ten batch alma
        data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]  # (32, 32, 3)
        input_data = np.random.random((8, *expected_shape))  # Batch boyutu 8 ile uyumlu
        data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"

    try:
        # Seçilen katmana kadar tüm katmanları çalıştır
        real_output = get_layer_output(loaded_model, layer_index, input_data)

        # FPGA çıktısını yükle
        if fpga_output_file is None:
            return "Lütfen FPGA çıktısını yükleyin!", "<p>FPGA çıktısı yüklenmedi.</p>"
        fpga_output = np.load(fpga_output_file.name)

        # Çıktıyı NumPy array'e çevir
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        # Benzerlik hesaplama (MSE ile)
        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)  # 0-1 arasında skor

        # Beklenen giriş şekli
        expected_input_shape = loaded_model.input.shape[1:] if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape[1:]

        # Sonuçları hazırla
        output_str = f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Donanıma Gönderilen Çıktı Boyutu: {len(serialized_output)} bayt\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        # Animasyonlu HTML çıktısı
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Katman bilgilerini dinamik olarak alma fonksiyonu
def update_layer_dropdown(model_selection):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    loaded_model = tf.keras.models.load_model(model_path)
    layer_names = [f"Katman {i}: {layer.name} - Çıkış Şekli: {layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'}"
                   for i, layer in enumerate(loaded_model.layers)]
    return gr.Dropdown(choices=layer_names, value=layer_names[0])

# Gradio arayüzünü oluştur
with gr.Blocks(title="LeNet CNN Katman Hızlandırıcı") as interface:
    gr.Markdown("# LeNet CNN Katman Hızlandırıcı")
    gr.Markdown("Modelinizi seçin, ardından katmanınızı ve giriş verisi tipini belirleyin. Katman girişini dışa aktarın ve FPGA çıktısını yükleyerek karşılaştırma yapın!")

    # Model seçimi
    with gr.Row():
        model_input = gr.Dropdown(choices=list(MODELS.keys()), label="Model Seçimi", value="LeNet1")

    # Katman seçimi ve simülasyon
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin")
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")

            # Girişi dışa aktarma
            export_btn = gr.Button("Katman Girişini Dışa Aktar", variant="secondary")
            input_text = gr.Textbox(label="Katman Giriş Bilgisi", lines=3)
            input_file = gr.File(label="Giriş Dosyası (Kağan'a Göndermek İçin İndirin)")

            # FPGA çıktısını yükleme
            fpga_output_file = gr.File(label="Kağan'dan Gelen FPGA Çıktısını Yükleyin (.npy formatında)")

            # Karşılaştırma butonu
            submit_btn = gr.Button("Karşılaştırma Yap", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Model seçildiğinde katman dropdown'unu güncelle
    model_input.change(fn=update_layer_dropdown, inputs=model_input, outputs=layer_input)

    # Girişi dışa aktar
    export_btn.click(fn=export_layer_input, inputs=[model_input, layer_input, data_input], outputs=[input_text, input_file])

    # Karşılaştırma yap
    submit_btn.click(fn=run_layer, inputs=[model_input, layer_input, data_input, fpga_output_file], outputs=[output_text, output_animation])

# Arayüzü başlat
interface.launch()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://92f4e5c37df3d138eb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from google.colab import drive
import gradio as gr
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import io

# Google Drive'ı bağla
drive.mount('/content/drive')

# Model ve veri yollarını tanımla
MODELS = {
    "LeNet1": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
        "flow_from_directory": True
    },
    "LeNet2": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
        "flow_from_directory": False
    },
    "LeNet3": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
        "flow_from_directory": False
    }
}

# Veri yükleme fonksiyonları
# Alt klasörsüz dizinler için
def load_images_from_directory(directory, target_size=(32, 32), batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0  # Normalizasyon
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

# Alt klasörlü dizinler için
def load_images_with_flow_from_directory(directory, target_size=(32, 32), batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Katman girişini dışa aktarma fonksiyonu
def export_layer_input(model_selection, layer_selection, data_choice):
    # Model ve veri yollarını al
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Modeli yükle
    loaded_model = tf.keras.models.load_model(model_path)

    # Test verisini yükle
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(32, 32), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(32, 32), batch_size=8)

    # Seçilen katman indeksini al
    layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))

    # Veri seçimi ve giriş hazırlanması
    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)  # ImageDataGenerator'dan batch alma
        else:
            input_data = next(iter(test_data))  # tf.data.Dataset'ten batch alma
        data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]  # (32, 32, 3)
        input_data = np.random.random((8, *expected_shape))  # Batch boyutu 8 ile uyumlu
        data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"

    # Eğer ilk katman değilse, önceki katmanların çıktısını giriş olarak al
    if layer_index > 0:
        intermediate_model = tf.keras.Model(inputs=loaded_model.input, outputs=loaded_model.layers[layer_index - 1].output)
        input_data = intermediate_model.predict(input_data, verbose=0)

    # Giriş verisini hexadecimal formatına çevir ve kaydet
    hex_filename = f"layer_{layer_index}_input_hex.txt"
    flat_data = input_data.flatten().astype(np.float32)  # Veriyi düzleştir ve float32'ye çevir
    # Veriyi baytlarına ayır ve her baytı "0xaa" formatında yaz
    hex_data = []
    for value in flat_data:
        bytes_value = np.float32(value).tobytes()  # float32'yi baytlara çevir
        hex_bytes = [f"0x{byte:02x}" for byte in bytes_value]  # Her baytı 0xaa formatına çevir
        hex_data.extend(hex_bytes)

    # Hex veriyi virgülle ayrılmış şekilde dosyaya yaz
    with open(hex_filename, 'w') as f:
        f.write(','.join(hex_data))  # 0xaa,0xbb,... formatında yaz

    # Giriş verisini metin olarak göster
    input_text = f"Katman Giriş Verisi Şekli: {input_data.shape}\n"
    input_text += f"Örnek Veri (Float): {flat_data[:5]}\n"
    input_text += f"Örnek Veri (Hex, ilk 5 bayt): {hex_data[:5]}\n"
    input_text += f"Giriş verisi hexadecimal formatta '{hex_filename}' dosyasına kaydedildi. Bu dosyayı indirip Kağan'a gönderebilirsiniz."

    return input_text, gr.File(value=hex_filename)

# Katman çıktısını alma fonksiyonu
def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# FPGA simülasyonu ve karşılaştırma
def run_layer(model_selection, layer_selection, data_choice, fpga_output_file):
    # Model ve veri yollarını al
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Modeli yükle
    loaded_model = tf.keras.models.load_model(model_path)

    # Test verisini yükle
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(32, 32), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(32, 32), batch_size=8)

    # Seçilen katman indeksini al
    layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
    current_layer = loaded_model.layers[layer_index]

    # Veri seçimi ve giriş hazırlanması
    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)  # ImageDataGenerator'dan batch alma
        else:
            input_data = next(iter(test_data))  # tf.data.Dataset'ten batch alma
        data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]  # (32, 32, 3)
        input_data = np.random.random((8, *expected_shape))  # Batch boyutu 8 ile uyumlu
        data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"

    try:
        # Seçilen katmana kadar tüm katmanları çalıştır
        real_output = get_layer_output(loaded_model, layer_index, input_data)

        # FPGA çıktısını yükle
        if fpga_output_file is None:
            return "Lütfen FPGA çıktısını yükleyin!", "<p>FPGA çıktısı yüklenmedi.</p>"
        fpga_output = np.load(fpga_output_file.name)

        # Çıktıyı NumPy array'e çevir
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        # Benzerlik hesaplama (MSE ile)
        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)  # 0-1 arasında skor

        # Beklenen giriş şekli
        expected_input_shape = loaded_model.input.shape[1:] if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape[1:]

        # Sonuçları hazırla
        output_str = f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Donanıma Gönderilen Çıktı Boyutu: {len(serialized_output)} bayt\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        # Animasyonlu HTML çıktısı
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Katman bilgilerini dinamik olarak alma fonksiyonu
def update_layer_dropdown(model_selection):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    loaded_model = tf.keras.models.load_model(model_path)
    layer_names = [f"Katman {i}: {layer.name} - Çıkış Şekli: {layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'}"
                   for i, layer in enumerate(loaded_model.layers)]
    return gr.Dropdown(choices=layer_names, value=layer_names[0])

# Gradio arayüzünü oluştur
with gr.Blocks(title="LeNet CNN Katman Hızlandırıcı") as interface:
    gr.Markdown("# LeNet CNN Katman Hızlandırıcı")
    gr.Markdown("Modelinizi seçin, ardından katmanınızı ve giriş verisi tipini belirleyin. Katman girişini dışa aktarın ve FPGA çıktısını yükleyerek karşılaştırma yapın!")

    # Model seçimi
    with gr.Row():
        model_input = gr.Dropdown(choices=list(MODELS.keys()), label="Model Seçimi", value="LeNet1")

    # Katman seçimi ve simülasyon
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin")
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")

            # Girişi dışa aktarma
            export_btn = gr.Button("Katman Girişini Dışa Aktar", variant="secondary")
            input_text = gr.Textbox(label="Katman Giriş Bilgisi", lines=3)
            input_file = gr.File(label="Hex Dosyası (Kağan'a Göndermek İçin İndirin)")

            # FPGA çıktısını yükleme
            fpga_output_file = gr.File(label="Kağan'dan Gelen FPGA Çıktısını Yükleyin (.npy formatında)")

            # Karşılaştırma butonu
            submit_btn = gr.Button("Karşılaştırma Yap", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Model seçildiğinde katman dropdown'unu güncelle
    model_input.change(fn=update_layer_dropdown, inputs=model_input, outputs=layer_input)

    # Girişi dışa aktar
    export_btn.click(fn=export_layer_input, inputs=[model_input, layer_input, data_input], outputs=[input_text, input_file])

    # Karşılaştırma yap
    submit_btn.click(fn=run_layer, inputs=[model_input, layer_input, data_input, fpga_output_file], outputs=[output_text, output_animation])

# Arayüzü başlat
interface.launch()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://204d1ba6da2c56826a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import tensorflow as tf    #Last Work
import numpy as np
from tensorflow.keras import layers

!pip install gradio
import gradio as gr

import os
import threading
import queue
import time
import json
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import uuid

# Define model and data paths
MODELS = {
    "LeNet1": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
        "flow_from_directory": True
    },
    "LeNet2": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
        "flow_from_directory": False
    },
    "LeNet3": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
        "flow_from_directory": False
    }
}

# Data loading functions
def load_images_from_directory(directory, target_size=(32, 32), batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

def load_images_with_flow_from_directory(directory, target_size=(32, 32), batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Extract weights and window parameters
def extract_weights_and_window(layer):
    weights = layer.get_weights()
    window_params = {}
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = layer.kernel_size
        window_params["stride"] = layer.strides
        window_params["padding"] = layer.padding
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units
    return weights, window_params

# Simulate FPGA communication
def simulate_fpga_communication(input_data, output_file, result_queue):
    time.sleep(1)  # Simulate communication delay
    with open(output_file, 'rb') as f:
        data = np.frombuffer(f.read(), dtype=np.float32)
    fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)
    result_queue.put((fpga_output, len(data)))

# Layer output function
def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Save data for Kagan in JSON format
def save_data_for_kagan(weights, window_params, data_size):
    data = {
        "weights": [w.flatten().tolist() for w in weights],  # Convert weights to list
        "window_params": window_params,
        "data_size_bytes": data_size
    }
    json_file = f"data_for_kagan_{uuid.uuid4()}.json"
    with open(json_file, 'w') as f:
        json.dump(data, f, indent=4)
    return json_file

# Main simulation function
def run_model(model_selection, layer_selection, data_choice, mode):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Load model
    loaded_model = tf.keras.models.load_model(model_path)

    # Load test data
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(32, 32), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(32, 32), batch_size=8)

    # Handle layer selection
    layer_index = None
    current_layer = None
    if layer_selection and layer_selection != "None":
        layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
        current_layer = loaded_model.layers[layer_index]

    # Data selection
    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)
        else:
            input_data = next(iter(test_data))
        data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]
        input_data = np.random.random((8, *expected_shape))
        data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"

    try:
        if layer_index is None or layer_selection == "None":
            # Run the entire model without acceleration
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            json_file = None
        else:
            # Run with acceleration
            weights, window_params = extract_weights_and_window(current_layer)
            real_output = get_layer_output(loaded_model, layer_index, input_data)

            # Thread 1: Simulation and file output
            output_file_x = f"file_{uuid.uuid4()}.x"
            output_file_y = f"file_{uuid.uuid4()}.y"
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())
            with open(output_file_y, 'wb') as f:
                for w in weights:
                    f.write(w.flatten().tobytes())

            # Save data for Kagan
            json_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()))

            # Thread 2: FPGA communication
            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=simulate_fpga_communication,
                args=(real_output, output_file_x, result_queue)
            )
            comm_thread.start()
            comm_thread.join()

            fpga_output, data_sent_size = result_queue.get()
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            # Clean up files
            os.remove(output_file_x)
            os.remove(output_file_y)
            if json_file:
                os.remove(json_file)

        # Convert outputs to NumPy
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        # Similarity calculation
        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        # Expected input shape
        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        # Prepare results
        output_str = f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        # Animated HTML output
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Update layer dropdown
def update_layer_dropdown(model_selection):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    loaded_model = tf.keras.models.load_model(model_path)
    layer_names = ["None"] + [f"Katman {i}: {layer.name} - Çıkış Şekli: {layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'}"
                              for i, layer in enumerate(loaded_model.layers)]
    return gr.Dropdown(choices=layer_names, value="None")

# Gradio interface
with gr.Blocks(title="LeNet CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# LeNet CNN Model Hızlandırıcı")
    gr.Markdown("Modelinizi seçin, katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")

    # Model and mode selection
    with gr.Row():
        model_input = gr.Dropdown(choices=list(MODELS.keys()), label="Model Seçimi", value="LeNet1")
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")

    # Layer selection and simulation
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin (None = Hızlandırma Yok)")
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Update layer dropdown when model is selected
    model_input.change(fn=update_layer_dropdown, inputs=model_input, outputs=layer_input)

    # Run the model
    submit_btn.click(fn=run_model, inputs=[model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Start the interface
interface.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.7/322.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.8 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e6ed0f8aeacc3ff16e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory

In [ ]:
import tensorflow as tf                      #TRY 1
import numpy as np
from tensorflow.keras import layers
import gradio as gr
import os
import threading
import queue
import time
!pip install pyserial
import serial
import struct
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import uuid

# Define model and data paths
MODELS = {
    "LeNet1": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
        "flow_from_directory": True
    },
    "LeNet2": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
        "flow_from_directory": False
    },
    "LeNet3": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
        "flow_from_directory": False
    }
}

# Data loading functions
def load_images_from_directory(directory, target_size=(32, 32), batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

def load_images_with_flow_from_directory(directory, target_size=(32, 32), batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Extract weights and window parameters
def extract_weights_and_window(layer):
    weights = layer.get_weights()
    window_params = {}
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = list(layer.kernel_size)
        window_params["stride"] = list(layer.strides)
        window_params["padding"] = layer.padding
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units
    return weights, window_params

# FPGA communication function (simulating Vivado-based UART communication)
def communicate_with_fpga(input_data, output_file, result_queue, mode):
    if mode == "Simülasyon":
        # Simulate FPGA communication
        time.sleep(1)  # Simulate delay
        with open(output_file, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.float32)
        fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)
        result_queue.put((fpga_output, len(data)))
    else:
        # Real FPGA communication via UART
        try:
            # Configure serial port (adjust COM port and baud rate as needed)
            ser = serial.Serial(
                port='COM3',  # Kağan'ın kullandığı portu belirtmesi gerekir
                baudrate=115200,
                timeout=5
            )

            # Read input data from file
            with open(output_file, 'rb') as f:
                data = f.read()

            # Send data to FPGA
            ser.write(data)
            ser.flush()

            # Receive response from FPGA (assuming same size as input)
            response_size = len(data)
            response = ser.read(response_size)

            # Close serial port
            ser.close()

            # Convert response to NumPy array
            fpga_output = np.frombuffer(response, dtype=np.float32).reshape(input_data.shape)
            result_queue.put((fpga_output, len(data)))

        except Exception as e:
            result_queue.put((None, 0, f"FPGA iletişimi hatası: {str(e)}"))

# Layer output function
def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Save data for Kagan in binary format
def save_data_for_kagan(weights, window_params, data_size, output_data):
    bin_file = f"data_for_kagan_{uuid.uuid4()}.bin"
    with open(bin_file, 'wb') as f:
        # Write weights
        for w in weights:
            f.write(w.flatten().astype(np.float32).tobytes())
        # Write window parameters (serialized as binary)
        if window_params:
            if "kernel_size" in window_params:
                f.write(struct.pack('ii', *window_params["kernel_size"]))  # 2 integers
                f.write(struct.pack('ii', *window_params["stride"]))  # 2 integers
                padding_val = 1 if window_params["padding"] == "valid" else 0
                f.write(struct.pack('i', padding_val))  # 1 integer
            elif "units" in window_params:
                f.write(struct.pack('i', window_params["units"]))  # 1 integer
        # Write data size
        f.write(struct.pack('Q', data_size))  # Unsigned long long for size
        # Write output data
        f.write(output_data.flatten().astype(np.float32).tobytes())
    return bin_file

# Main simulation function
def run_model(model_selection, layer_selection, data_choice, mode):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Load model
    loaded_model = tf.keras.models.load_model(model_path)

    # Load test data
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(32, 32), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(32, 32), batch_size=8)

    # Handle layer selection
    layer_index = None
    current_layer = None
    if layer_selection and layer_selection != "None":
        layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
        current_layer = loaded_model.layers[layer_index]

    # Data selection
    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)
        else:
            input_data = next(iter(test_data))
        data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]
        input_data = np.random.random((8, *expected_shape))
        data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"

    try:
        if layer_index is None or layer_selection == "None":
            # Run the entire model without acceleration
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            bin_file = None
        else:
            # Run with acceleration
            weights, window_params = extract_weights_and_window(current_layer)
            real_output = get_layer_output(loaded_model, layer_index, input_data)

            # Thread 1: Simulation and file output
            output_file_x = f"file_{uuid.uuid4()}.x"
            output_file_y = f"file_{uuid.uuid4()}.y"
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())
            with open(output_file_y, 'wb') as f:
                for w in weights:
                    f.write(w.flatten().tobytes())

            # Save data for Kagan in binary format
            bin_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()), real_output)

            # Thread 2: FPGA communication
            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=communicate_with_fpga,
                args=(real_output, output_file_x, result_queue, mode)
            )
            comm_thread.start()
            comm_thread.join()

            fpga_result = result_queue.get()
            if len(fpga_result) == 3:  # Error case
                raise Exception(fpga_result[2])
            fpga_output, data_sent_size = fpga_result
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            # Clean up files
            os.remove(output_file_x)
            os.remove(output_file_y)
            if bin_file:
                os.remove(bin_file)

        # Convert outputs to NumPy
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        # Similarity calculation
        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        # Expected input shape
        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        # Prepare results
        output_str = f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        # Animated HTML output
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Update layer dropdown
def update_layer_dropdown(model_selection):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    loaded_model = tf.keras.models.load_model(model_path)
    layer_names = ["None"] + [f"Katman {i}: {layer.name} - Çıkış Şekli: {layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'}"
                              for i, layer in enumerate(loaded_model.layers)]
    return gr.Dropdown(choices=layer_names, value="None")

# Gradio interface
with gr.Blocks(title="LeNet CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# LeNet CNN Model Hızlandırıcı")
    gr.Markdown("Modelinizi seçin, katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")

    # Model and mode selection
    with gr.Row():
        model_input = gr.Dropdown(choices=list(MODELS.keys()), label="Model Seçimi", value="LeNet1")
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")

    # Layer selection and simulation
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin (None = Hızlandırma Yok)")
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Update layer dropdown when model is selected
    model_input.change(fn=update_layer_dropdown, inputs=model_input, outputs=layer_input)

    # Run the model
    submit_btn.click(fn=run_model, inputs=[model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Start the interface
interface.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.9 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6b9132583128d7bb3c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#TRY 2
!pip install pyserial
!pip install gradio
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
import gradio as gr
import os
import threading
import queue
import time
import serial
import struct
import serial.tools.list_ports
import platform
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import uuid


# Define model and data paths
MODELS = {
    "LeNet1": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
        "flow_from_directory": True
    },
    "LeNet2": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
        "flow_from_directory": False
    },
    "LeNet3": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
        "flow_from_directory": False
    }
}

# Data loading functions
def load_images_from_directory(directory, target_size=(32, 32), batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

def load_images_with_flow_from_directory(directory, target_size=(32, 32), batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Extract weights and window parameters
def extract_weights_and_window(layer):
    weights = layer.get_weights()
    window_params = {}
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = list(layer.kernel_size)
        window_params["stride"] = list(layer.strides)
        window_params["padding"] = layer.padding
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units
    return weights, window_params

# Get available serial ports
def get_available_ports():
    ports = serial.tools.list_ports.comports()
    return [port.device for port in ports]

# FPGA communication function (UART for Nexys4DDR)
def communicate_with_fpga(input_data, output_file, result_queue, mode):
    if mode == "Simülasyon":
        # Simulate FPGA communication
        time.sleep(1)  # Simulate delay
        with open(output_file, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.float32)
        fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)
        result_queue.put((fpga_output, len(data)))
    else:
        # Real FPGA communication via UART for Nexys4DDR
        try:
            # Select port based on platform
            default_port = "COM3" if platform.system() == "Windows" else "/dev/ttyUSB0"
            available_ports = get_available_ports()
            if default_port not in available_ports:
                raise Exception(f"Port {default_port} bulunamadı. Mevcut portlar: {available_ports or 'Yok'}")

            # Configure serial port for Nexys4DDR
            ser = serial.Serial(
                port=default_port,
                baudrate=115200,
                stopbits=serial.STOPBITS_TWO,
                bytesize=serial.EIGHTBITS,
                parity=serial.PARITY_NONE,
                timeout=5
            )

            # Read input data from file
            with open(output_file, 'rb') as f:
                data = f.read()

            # Send data to FPGA
            ser.write(data)
            ser.flush()

            # Receive response from FPGA (assuming same size as input)
            response_size = len(data)
            response = ser.read(response_size)

            # Close serial port
            ser.close()

            if len(response) != response_size:
                raise Exception(f"FPGA'dan eksik veri alındı: {len(response)}/{response_size} bayt")

            # Convert response to NumPy array
            fpga_output = np.frombuffer(response, dtype=np.float32).reshape(input_data.shape)
            result_queue.put((fpga_output, len(data)))

        except Exception as e:
            available_ports = get_available_ports()
            error_msg = f"FPGA iletişimi hatası: {str(e)}\nMevcut portlar: {available_ports or 'Yok'}\nLütfen doğru portu kontrol edin veya FPGA'nın bağlı olduğundan emin olun."
            result_queue.put((None, 0, error_msg))

# Layer output function
def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Save data for Kagan in binary format
def save_data_for_kagan(weights, window_params, data_size, output_data):
    bin_file = f"data_for_kagan_{uuid.uuid4()}.bin"
    with open(bin_file, 'wb') as f:
        # Write weights
        for w in weights:
            f.write(w.flatten().astype(np.float32).tobytes())
        # Write window parameters (Kağan'ın sırasına göre)
        if window_params:
            if "kernel_size" in window_params:
                f.write(struct.pack('ii', *window_params["kernel_size"]))  # 2 integers
                f.write(struct.pack('ii', *window_params["stride"]))  # 2 integers
                padding_val = 1 if window_params["padding"] == "valid" else 0
                f.write(struct.pack('i', padding_val))  # 1 integer
            elif "units" in window_params:
                f.write(struct.pack('i', window_params["units"]))  # 1 integer
        # Write data size
        f.write(struct.pack('Q', data_size))  # Unsigned long long for size
        # Write output data
        f.write(output_data.flatten().astype(np.float32).tobytes())
    return bin_file

# Main simulation function
def run_model(model_selection, layer_selection, data_choice, mode):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Load model
    loaded_model = tf.keras.models.load_model(model_path)

    # Load test data
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(32, 32), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(32, 32), batch_size=8)

    # Handle layer selection
    layer_index = None
    current_layer = None
    if layer_selection and layer_selection != "None":
        layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
        current_layer = loaded_model.layers[layer_index]

    # Data selection
    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)
        else:
            input_data = next(iter(test_data))
        data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]
        input_data = np.random.random((8, *expected_shape))
        data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"

    try:
        if layer_index is None or layer_selection == "None":
            # Run the entire model without acceleration
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            bin_file = None
        else:
            # Run with acceleration
            weights, window_params = extract_weights_and_window(current_layer)
            real_output = get_layer_output(loaded_model, layer_index, input_data)

            # Thread 1: Simulation and file output
            output_file_x = f"file_{uuid.uuid4()}.x"
            output_file_y = f"file_{uuid.uuid4()}.y"
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())
            with open(output_file_y, 'wb') as f:
                for w in weights:
                    f.write(w.flatten().tobytes())

            # Save data for Kagan in binary format
            bin_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()), real_output)

            # Thread 2: FPGA communication
            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=communicate_with_fpga,
                args=(real_output, output_file_x, result_queue, mode)
            )
            comm_thread.start()
            comm_thread.join()

            fpga_result = result_queue.get()
            if len(fpga_result) == 3:  # Error case
                raise Exception(fpga_result[2])
            fpga_output, data_sent_size = fpga_result
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            # Clean up files
            os.remove(output_file_x)
            os.remove(output_file_y)
            if bin_file:
                os.remove(bin_file)

        # Convert outputs to NumPy
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        # Similarity calculation
        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        # Expected input shape
        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        # Prepare results
        output_str = f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        # Animated HTML output
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Update layer dropdown
def update_layer_dropdown(model_selection):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    loaded_model = tf.keras.models.load_model(model_path)
    layer_names = ["None"] + [f"Katman {i}: {layer.name} - Çıkış Şekli: {layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'}"
                              for i, layer in enumerate(loaded_model.layers)]
    return gr.Dropdown(choices=layer_names, value="None")

# Gradio interface
with gr.Blocks(title="LeNet CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# LeNet CNN Model Hızlandırıcı")
    gr.Markdown("Modelinizi seçin, katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")

    # Model and mode selection
    with gr.Row():
        model_input = gr.Dropdown(choices=list(MODELS.keys()), label="Model Seçimi", value="LeNet1")
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")

    # Layer selection and simulation
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin (None = Hızlandırma Yok)")
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Update layer dropdown when model is selected
    model_input.change(fn=update_layer_dropdown, inputs=model_input, outputs=layer_input)

    # Run the model
    submit_btn.click(fn=run_model, inputs=[model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Start the interface
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f2e9bb57bda6b6e35f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
  #Last CODE
# Gerekli kütüphaneleri içe aktar
!pip install pyserial
!pip install gradio
import tensorflow as tf  # Sinir ağı modeli için
import numpy as np  # Sayısal işlemler için
from tensorflow.keras import layers  # Keras katmanları için
import gradio as gr  # Kullanıcı arayüzü için
import os  # Dosya işlemleri için
import threading  # Çoklu iş parçacığı için
import queue  # İş parçacıkları arası veri aktarımı için
import time  # Simülasyon gecikmesi için
import serial  # UART iletişimi için
import struct  # Binary veri paketleme için
import serial.tools.list_ports  # Mevcut seri portları listelemek için
import platform  # İşletim sistemi bilgisi için
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Resim yükleme ve işleme için
import uuid  # Benzersiz dosya adları için

# Model ve veri yollarını tanımla
MODELS = {
    "LeNet1": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",  # Model dosya yolu
        "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",  # Test veri yolu
        "flow_from_directory": True  # Alt klasörlerden veri yükleme
    },
    "LeNet2": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
        "flow_from_directory": False
    },
    "LeNet3": {
        "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
        "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
        "flow_from_directory": False
    }
}

# Veri yükleme fonksiyonu: Alt klasörsüz dizinlerden resimleri yükler
def load_images_from_directory(directory, target_size=(32, 32), batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)  # Resmi belirtilen boyutta yükle
            img_array = img_to_array(img) / 255.0  # Resmi diziye çevir ve normalize et
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)  # Veriyi toplu işleme uygun hale getir

# Veri yükleme fonksiyonu: Alt klasörlü dizinlerden resimleri yükler
def load_images_with_flow_from_directory(directory, target_size=(32, 32), batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)  # Normalizasyon için veri üreteci
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data  # ImageDataGenerator nesnesi döndür

# Ağırlık ve pencere parametrelerini çıkarma fonksiyonu
def extract_weights_and_window(layer):
    weights = layer.get_weights()  # Katmanın ağırlıklarını al
    window_params = {}  # Pencere parametrelerini saklamak için sözlük
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = list(layer.kernel_size)  # Çekirdek boyutu (ör. [5, 5])
        window_params["stride"] = list(layer.strides)  # Adım boyutu (ör. [1, 1])
        window_params["padding"] = layer.padding  # Dolgu tipi (valid veya same)
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units  # Birim sayısı
    return weights, window_params  # Ağırlıklar ve parametreler döndür

# Mevcut seri portları listeleyen fonksiyon
def get_available_ports():
    ports = serial.tools.list_ports.comports()  # Sistemdeki tüm seri portları al
    return [port.device for port in ports]  # Port adlarını listele

# FPGA ile iletişim fonksiyonu (Nexys4DDR için UART)
def communicate_with_fpga(input_data, output_file, result_queue, mode):
    if mode == "Simülasyon":
        # Simülasyon modu: FPGA yerine gürültü eklenmiş veri döndür
        time.sleep(1)  # İletişim gecikmesini taklit et
        with open(output_file, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.float32)  # Dosyadan veriyi oku
        fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)  # Gürültü ekle
        result_queue.put((fpga_output, len(data)))  # Sonucu kuyruğa ekle
    else:
        # Donanım modu: Nexys4DDR ile UART üzerinden gerçek iletişim
        try:
            # Platforma göre varsayılan port seç
            default_port = "COM3" if platform.system() == "Windows" else "/dev/ttyUSB0"
            available_ports = get_available_ports()
            if default_port not in available_ports:
                raise Exception(f"Port {default_port} bulunamadı. Mevcut portlar: {available_ports or 'Yok'}")

            # Seri portu yapılandır (Nexys4DDR için UART ayarları)
            ser = serial.Serial(
                port=default_port,
                baudrate=115200,  # 115200 baud rate
                stopbits=serial.STOPBITS_TWO,  # 2 stop bit
                bytesize=serial.EIGHTBITS,  # 8 veri biti
                parity=serial.PARITY_NONE,  # Parity yok
                timeout=5  # 5 saniye zaman aşımı
            )

            # Dosyadan veriyi oku
            with open(output_file, 'rb') as f:
                data = f.read()

            # Veriyi FPGA'ya gönder
            ser.write(data)
            ser.flush()  # Gönderim tamamlanana kadar bekle

            # FPGA'dan yanıtı al (aynı boyutta veri bekleniyor)
            response_size = len(data)
            response = ser.read(response_size)

            # Seri portu kapat
            ser.close()

            # Yanıtın doğruluğunu kontrol et
            if len(response) != response_size:
                raise Exception(f"FPGA'dan eksik veri alındı: {len(response)}/{response_size} bayt")

            # Yanıtı NumPy dizisine çevir
            fpga_output = np.frombuffer(response, dtype=np.float32).reshape(input_data.shape)
            result_queue.put((fpga_output, len(data)))

        except Exception as e:
            # Hata durumunda, mevcut portları listele ve açıklayıcı mesaj döndür
            available_ports = get_available_ports()
            error_msg = f"FPGA iletişimi hatası: {str(e)}\nMevcut portlar: {available_ports or 'Yok'}\nLütfen doğru portu kontrol edin veya FPGA'nın bağlı olduğundan emin olun."
            result_queue.put((None, 0, error_msg))

# Katman çıktısını hesaplama fonksiyonu
def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        # İlk katman için: Modelin girişinden ilk katman çıkışına kadar hesapla
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        # Diğer katmanlar için: Önceki katmanların çıktısını adım adım hesapla
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Kağan için veriyi binary formatta kaydetme fonksiyonu
def save_data_for_kagan(weights, window_params, data_size, output_data):
    bin_file = f"data_for_kagan_{uuid.uuid4()}.bin"  # Benzersiz dosya adı
    with open(bin_file, 'wb') as f:
        # Ağırlıkları yaz (Kağan'ın sırasına göre önce ağırlıklar)
        for w in weights:
            f.write(w.flatten().astype(np.float32).tobytes())  # float32 formatında
        # Pencere parametrelerini yaz
        if window_params:
            if "kernel_size" in window_params:
                f.write(struct.pack('ii', *window_params["kernel_size"]))  # 2 tamsayı (8 bayt)
                f.write(struct.pack('ii', *window_params["stride"]))  # 2 tamsayı (8 bayt)
                padding_val = 1 if window_params["padding"] == "valid" else 0
                f.write(struct.pack('i', padding_val))  # 1 tamsayı (4 bayt)
            elif "units" in window_params:
                f.write(struct.pack('i', window_params["units"]))  # 1 tamsayı (4 bayt)
        # Veri boyutunu yaz
        f.write(struct.pack('Q', data_size))  # Unsigned long long (8 bayt)
        # Çıkış verilerini yaz
        f.write(output_data.flatten().astype(np.float32).tobytes())  # float32 formatında
    return bin_file  # Oluşturulan dosyanın adını döndür

# Ana simülasyon ve çalıştırma fonksiyonu
def run_model(model_selection, layer_selection, data_choice, mode):
    model_info = MODELS[model_selection]  # Seçilen modelin bilgilerini al
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Modeli yükle
    loaded_model = tf.keras.models.load_model(model_path)

    # Test verisini yükle
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(32, 32), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(32, 32), batch_size=8)

    # Katman seçimini işle
    layer_index = None
    current_layer = None
    if layer_selection and layer_selection != "None":
        layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
        current_layer = loaded_model.layers[layer_index]

    # Veri seçimini yap
    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)  # ImageDataGenerator'dan bir grup veri al
        else:
            input_data = next(iter(test_data))  # tf.data.Dataset'ten bir grup veri al
        data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]  # Modelin beklediği giriş şekli
        input_data = np.random.random((8, *expected_shape))  # Rastgele veri oluştur
        data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"

    try:
        if layer_index is None or layer_selection == "None":
            # Hızlandırma yok: Modelin tamamını çalıştır
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output  # FPGA kullanılmıyor
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            bin_file = None
        else:
            # Hızlandırma ile çalış: Seçilen katmanı FPGA'da işle
            weights, window_params = extract_weights_and_window(current_layer)  # Ağırlık ve parametreleri al
            real_output = get_layer_output(loaded_model, layer_index, input_data)  # Katman çıktısını hesapla

            # Thread 1: Simülasyon ve dosya çıktısı
            output_file_x = f"file_{uuid.uuid4()}.x"  # Çıkış verileri için dosya
            output_file_y = f"file_{uuid.uuid4()}.y"  # Ağırlıklar için dosya
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())  # Çıkış verilerini yaz
            with open(output_file_y, 'wb') as f:
                for w in weights:
                    f.write(w.flatten().tobytes())  # Ağırlıkları yaz

            # Kağan için veriyi binary formatta kaydet
            bin_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()), real_output)

            # Thread 2: FPGA iletişimi
            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=communicate_with_fpga,
                args=(real_output, output_file_x, result_queue, mode)
            )
            comm_thread.start()
            comm_thread.join()

            # FPGA sonucunu al
            fpga_result = result_queue.get()
            if len(fpga_result) == 3:  # Hata durumu
                raise Exception(fpga_result[2])
            fpga_output, data_sent_size = fpga_result
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            # Dosyaları temizle
            os.remove(output_file_x)
            os.remove(output_file_y)
            if bin_file:
                os.remove(bin_file)

        # Çıktıları NumPy formatına çevir
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        # Benzerlik skorunu hesapla (MSE ile)
        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        # Beklenen giriş şeklini belirle
        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        # Sonuçları hazırla
        output_str = f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        # Animasyonlu HTML çıktısı
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        # Hata durumunda açıklayıcı mesaj döndür
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Katman seçim dropdown'unu güncelleme fonksiyonu
def update_layer_dropdown(model_selection):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    loaded_model = tf.keras.models.load_model(model_path)  # Modeli yükle
    layer_names = ["None"] + [f"Katman {i}: {layer.name} - Çıkış Şekli: {layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'}"
                              for i, layer in enumerate(loaded_model.layers)]  # Katman adlarını ve şekillerini listele
    return gr.Dropdown(choices=layer_names, value="None")  # Dropdown nesnesi döndür

# Gradio arayüzünü oluştur
with gr.Blocks(title="LeNet CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# LeNet CNN Model Hızlandırıcı")  # Başlık
    gr.Markdown("Modelinizi seçin, katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")  # Açıklama

    # Model ve mod seçimi
    with gr.Row():
        model_input = gr.Dropdown(choices=list(MODELS.keys()), label="Model Seçimi", value="LeNet1")  # Model seçimi dropdown
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")  # Simülasyon/Donanım radyo düğmesi

    # Katman seçimi ve simülasyon
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin (None = Hızlandırma Yok)")  # Katman seçimi dropdown
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")  # Veri seçimi radyo düğmesi
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")  # Çalıştırma düğmesi

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)  # Sonuçları gösteren metin kutusu
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")  # Animasyonlu çıktı

    # Model seçildiğinde katman dropdown'unu güncelle
    model_input.change(fn=update_layer_dropdown, inputs=model_input, outputs=layer_input)

    # Modeli çalıştır
    submit_btn.click(fn=run_model, inputs=[model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Arayüzü başlat
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://09df503d8095f222a6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Gerekli kütüphaneleri içe aktar  #LAST CODE2
!pip install pyserial
!pip install gradio
import tensorflow as tf  # Sinir ağı modeli için
import numpy as np  # Sayısal işlemler için
from tensorflow.keras import layers  # Keras katmanları için
import gradio as gr  # Kullanıcı arayüzü için
import os  # Dosya işlemleri için
import threading  # Çoklu iş parçacığı için
import queue  # İş parçacıkları arası veri aktarımı için
import time  # Simülasyon gecikmesi için
import serial  # UART iletişimi için
import struct  # Binary veri paketleme için
import serial.tools.list_ports  # Mevcut seri portları listelemek için
import platform  # İşletim sistemi bilgisi için
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Resim yükleme ve işleme için
import uuid  # Benzersiz dosya adları için

# Model ve veri yollarını tanımla
MODELS = {
    "MobilNet1": {
        "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet1/saved_model/mobilenet_model.keras",  # Model dosya yolu
        "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",  # Test veri yolu
        "flow_from_directory": True  # Alt klasörlerden veri yükleme
    },
    "MobilNet2": {
        "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet2/saved_model/mobilenet_model.keras",
        "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
        "flow_from_directory": False
    },
    "MobilNet3": {
        "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet3/saved_model/mobilenet_model.keras",
        "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
        "flow_from_directory": False
    }
}

# Veri yükleme fonksiyonu: Alt klasörsüz dizinlerden resimleri yükler
def load_images_from_directory(directory, target_size=(224, 224), batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)  # Resmi belirtilen boyutta yükle
            img_array = img_to_array(img) / 255.0  # Resmi diziye çevir ve normalize et
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)  # Veriyi toplu işleme uygun hale getir

# Veri yükleme fonksiyonu: Alt klasörlü dizinlerden resimleri yükler
def load_images_with_flow_from_directory(directory, target_size=(224, 224), batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)  # Normalizasyon için veri üreteci
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data  # ImageDataGenerator nesnesi döndür

# Ağırlık ve pencere parametrelerini çıkarma fonksiyonu
def extract_weights_and_window(layer):
    weights = layer.get_weights()  # Katmanın ağırlıklarını al
    window_params = {}  # Pencere parametrelerini saklamak için sözlük
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = list(layer.kernel_size)  # Çekirdek boyutu
        window_params["stride"] = list(layer.strides)  # Adım boyutu
        window_params["padding"] = layer.padding  # Dolgu tipi
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units  # Birim sayısı
    return weights, window_params  # Ağırlıklar ve parametreler döndür

# Mevcut seri portları listeleyen fonksiyon
def get_available_ports():
    ports = serial.tools.list_ports.comports()  # Sistemdeki tüm seri portları al
    return [port.device for port in ports]  # Port adlarını listele

# FPGA ile iletişim fonksiyonu (Nexys4DDR için UART)
def communicate_with_fpga(input_data, output_file, result_queue, mode):
    if mode == "Simülasyon":
        # Simülasyon modu: FPGA yerine gürültü eklenmiş veri döndür
        time.sleep(1)  # İletişim gecikmesini taklit et
        with open(output_file, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.float32)  # Dosyadan veriyi oku
        fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)  # Gürültü ekle
        result_queue.put((fpga_output, len(data)))  # Sonucu kuyruğa ekle
    else:
        # Donanım modu: Nexys4DDR ile UART üzerinden gerçek iletişim
        try:
            # Platforma göre varsayılan port seç
            default_port = "COM3" if platform.system() == "Windows" else "/dev/ttyUSB0"
            available_ports = get_available_ports()
            if default_port not in available_ports:
                raise Exception(f"Port {default_port} bulunamadı. Mevcut portlar: {available_ports or 'Yok'}")

            # Seri portu yapılandır
            ser = serial.Serial(
                port=default_port,
                baudrate=115200,
                stopbits=serial.STOPBITS_TWO,
                bytesize=serial.EIGHTBITS,
                parity=serial.PARITY_NONE,
                timeout=5
            )

            # Dosyadan veriyi oku
            with open(output_file, 'rb') as f:
                data = f.read()

            # Veriyi FPGA'ya gönder
            ser.write(data)
            ser.flush()

            # FPGA'dan yanıtı al
            response_size = len(data)
            response = ser.read(response_size)

            # Seri portu kapat
            ser.close()

            # Yanıtın doğruluğunu kontrol et
            if len(response) != response_size:
                raise Exception(f"FPGA'dan eksik veri alındı: {len(response)}/{response_size} bayt")

            # Yanıtı NumPy dizisine çevir
            fpga_output = np.frombuffer(response, dtype=np.float32).reshape(input_data.shape)
            result_queue.put((fpga_output, len(data)))

        except Exception as e:
            # Hata durumunda açıklayıcı mesaj döndür
            available_ports = get_available_ports()
            error_msg = f"FPGA iletişimi hatası: {str(e)}\nMevcut portlar: {available_ports or 'Yok'}\nLütfen doğru portu kontrol edin veya FPGA'nın bağlı olduğundan emin olun."
            result_queue.put((None, 0, error_msg))

# Katman çıktısını hesaplama fonksiyonu
def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Kağan için veriyi binary formatta kaydetme fonksiyonu
def save_data_for_kagan(weights, window_params, data_size, output_data):
    bin_file = f"data_for_kagan_{uuid.uuid4()}.bin"
    with open(bin_file, 'wb') as f:
        for w in weights:
            f.write(w.flatten().astype(np.float32).tobytes())
        if window_params:
            if "kernel_size" in window_params:
                f.write(struct.pack('ii', *window_params["kernel_size"]))
                f.write(struct.pack('ii', *window_params["stride"]))
                padding_val = 1 if window_params["padding"] == "valid" else 0
                f.write(struct.pack('i', padding_val))
            elif "units" in window_params:
                f.write(struct.pack('i', window_params["units"]))
        f.write(struct.pack('Q', data_size))
        f.write(output_data.flatten().astype(np.float32).tobytes())
    return bin_file

# Ana simülasyon ve çalıştırma fonksiyonu
def run_model(model_selection, layer_selection, data_choice, mode):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Modeli yükle
    loaded_model = tf.keras.models.load_model(model_path)

    # Test verisini yükle
    if flow_from_directory:
        test_data = load_images_with_flow_from_directory(test_path, target_size=(224, 224), batch_size=8)
    else:
        test_data = load_images_from_directory(test_path, target_size=(224, 224), batch_size=8)

    # Katman seçimini işle
    layer_index = None
    current_layer = None
    if layer_selection and layer_selection != "None":
        layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
        current_layer = loaded_model.layers[layer_index]

    # Veri seçimini yap
    if data_choice == "Test Verisinden Örnek Kullan":
        if flow_from_directory:
            input_data = next(test_data)
        else:
            input_data = next(iter(test_data))
        data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
    else:
        expected_shape = loaded_model.input.shape[1:]
        input_data = np.random.random((8, *expected_shape))
        data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"

    try:
        if layer_index is None or layer_selection == "None":
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            bin_file = None
        else:
            weights, window_params = extract_weights_and_window(current_layer)
            real_output = get_layer_output(loaded_model, layer_index, input_data)

            output_file_x = f"file_{uuid.uuid4()}.x"
            output_file_y = f"file_{uuid.uuid4()}.y"
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())
            with open(output_file_y, 'wb') as f:
                for w in weights:
                    f.write(w.flatten().tobytes())

            bin_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()), real_output)

            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=communicate_with_fpga,
                args=(real_output, output_file_x, result_queue, mode)
            )
            comm_thread.start()
            comm_thread.join()

            fpga_result = result_queue.get()
            if len(fpga_result) == 3:
                raise Exception(fpga_result[2])
            fpga_output, data_sent_size = fpga_result
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            os.remove(output_file_x)
            os.remove(output_file_y)
            if bin_file:
                os.remove(bin_file)

        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        output_str = f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Katman seçim dropdown'unu güncelleme fonksiyonu
def update_layer_dropdown(model_selection):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    loaded_model = tf.keras.models.load_model(model_path)
    layer_names = ["None"] + [f"Katman {i}: {layer.name} - Çıkış Şekli: {layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'}"
                              for i, layer in enumerate(loaded_model.layers)]
    return gr.Dropdown(choices=layer_names, value="None")

# Gradio arayüzünü oluştur
with gr.Blocks(title="MobileNet CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# MobileNet CNN Model Hızlandırıcı")  # Başlık
    gr.Markdown("Modelinizi seçin, katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")  # Açıklama

    # Model ve mod seçimi
    with gr.Row():
        model_input = gr.Dropdown(choices=list(MODELS.keys()), label="Model Seçimi", value="MobilNet1")
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")

    # Katman seçimi ve simülasyon
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin (None = Hızlandırma Yok)")
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Model seçildiğinde katman dropdown'unu güncelle
    model_input.change(fn=update_layer_dropdown, inputs=model_input, outputs=layer_input)

    # Modeli çalıştır
    submit_btn.click(fn=run_model, inputs=[model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Arayüzü başlat
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://27a99f745e2494cbd7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# LAST CODE3

# Gerekli kütüphaneleri içe aktar
!pip install pyserial
!pip install gradio
import tensorflow as tf  # Sinir ağı modeli için
import numpy as np  # Sayısal işlemler için
from tensorflow.keras import layers  # Keras katmanları için
import gradio as gr  # Kullanıcı arayüzü için
import os  # Dosya işlemleri için
import threading  # Çoklu iş parçacığı için
import queue  # İş parçacıkları arası veri aktarımı için
import time  # Simülasyon gecikmesi için
import serial  # UART iletişimi için
import struct  # Binary veri paketleme için
import serial.tools.list_ports  # Mevcut seri portları listelemek için
import platform  # İşletim sistemi bilgisi için
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Resim yükleme ve işleme için
import uuid  # Benzersiz dosya adları için
from google.colab import drive  # Google Drive bağlantısı için

# Google Drive'ı bağla
drive.mount('/content/drive')

# Model ve veri yollarını tanımla
MODELS = {
    "VGG16-1": {
        "model_path": "/content/drive/MyDrive/Modeller/vgg16model-lenetsets/saved_model/vgg16_model.keras",  # Model dosya yolu
        "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",  # Test veri yolu
        "flow_from_directory": True  # Alt klasörlerden veri yükleme
    },
    "VGG16-2": {
        "model_path": "/content/drive/MyDrive/Modeller/vgg16model-vgg16sets/saved_model/vgg16_model.keras",
        "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
        "flow_from_directory": False
    },
    "VGG16-3": {
        "model_path": "/content/drive/MyDrive/Modeller/vgg16model-mobilnetsets/saved_model/vgg16_model.keras",
        "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
        "flow_from_directory": False
    }
}

# Veri yükleme fonksiyonu: Alt klasörsüz dizinlerden resimleri yükler
def load_images_from_directory(directory, target_size=(224, 224), batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)  # Resmi belirtilen boyutta yükle
            img_array = img_to_array(img) / 255.0  # Resmi diziye çevir ve normalize et
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)  # Veriyi toplu işleme uygun hale getir

# Veri yükleme fonksiyonu: Alt klasörlü dizinlerden resimleri yükler
def load_images_with_flow_from_directory(directory, target_size=(224, 224), batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)  # Normalizasyon için veri üreteci
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data  # ImageDataGenerator nesnesi döndür

# Ağırlık ve pencere parametrelerini çıkarma fonksiyonu
def extract_weights_and_window(layer):
    weights = layer.get_weights()  # Katmanın ağırlıklarını al
    window_params = {}  # Pencere parametrelerini saklamak için sözlük
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = list(layer.kernel_size)  # Çekirdek boyutu
        window_params["stride"] = list(layer.strides)  # Adım boyutu
        window_params["padding"] = layer.padding  # Dolgu tipi
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units  # Birim sayısı
    return weights, window_params  # Ağırlıklar ve parametreler döndür

# Mevcut seri portları listeleyen fonksiyon
def get_available_ports():
    ports = serial.tools.list_ports.comports()  # Sistemdeki tüm seri portları al
    return [port.device for port in ports]  # Port adlarını listele

# FPGA ile iletişim fonksiyonu (Nexys4DDR için UART)
def communicate_with_fpga(input_data, output_file, result_queue, mode):
    if mode == "Simülasyon":
        # Simülasyon modu: FPGA yerine gürültü eklenmiş veri döndür
        time.sleep(1)  # İletişim gecikmesini taklit et
        with open(output_file, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.float32)  # Dosyadan veriyi oku
        fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)  # Gürültü ekle
        result_queue.put((fpga_output, len(data)))  # Sonucu kuyruğa ekle
    else:
        # Donanım modu: Nexys4DDR ile UART üzerinden gerçek iletişim
        try:
            # Platforma göre varsayılan port seç
            default_port = "COM3" if platform.system() == "Windows" else "/dev/ttyUSB0"
            available_ports = get_available_ports()
            if default_port not in available_ports:
                raise Exception(f"Port {default_port} bulunamadı. Mevcut portlar: {available_ports or 'Yok'}")

            # Seri portu yapılandır
            ser = serial.Serial(
                port=default_port,
                baudrate=115200,
                stopbits=serial.STOPBITS_TWO,
                bytesize=serial.EIGHTBITS,
                parity=serial.PARITY_NONE,
                timeout=5
            )

            # Dosyadan veriyi oku
            with open(output_file, 'rb') as f:
                data = f.read()

            # Veriyi FPGA'ya gönder
            ser.write(data)
            ser.flush()

            # FPGA'dan yanıtı al
            response_size = len(data)
            response = ser.read(response_size)

            # Seri portu kapat
            ser.close()

            # Yanıtın doğruluğunu kontrol et
            if len(response) != response_size:
                raise Exception(f"FPGA'dan eksik veri alındı: {len(response)}/{response_size} bayt")

            # Yanıtı NumPy dizisine çevir
            fpga_output = np.frombuffer(response, dtype=np.float32).reshape(input_data.shape)
            result_queue.put((fpga_output, len(data)))

        except Exception as e:
            # Hata durumunda açıklayıcı mesaj döndür
            available_ports = get_available_ports()
            error_msg = f"FPGA iletişimi hatası: {str(e)}\nMevcut portlar: {available_ports or 'Yok'}\nLütfen doğru portu kontrol edin veya FPGA'nın bağlı olduğundan emin olun."
            result_queue.put((None, 0, error_msg))

# Katman çıktısını hesaplama fonksiyonu
def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Kağan için veriyi binary formatta kaydetme fonksiyonu
def save_data_for_kagan(weights, window_params, data_size, output_data):
    bin_file = f"data_for_kagan_{uuid.uuid4()}.bin"
    with open(bin_file, 'wb') as f:
        for w in weights:
            f.write(w.flatten().astype(np.float32).tobytes())
        if window_params:
            if "kernel_size" in window_params:
                f.write(struct.pack('ii', *window_params["kernel_size"]))
                f.write(struct.pack('ii', *window_params["stride"]))
                padding_val = 1 if window_params["padding"] == "valid" else 0
                f.write(struct.pack('i', padding_val))
            elif "units" in window_params:
                f.write(struct.pack('i', window_params["units"]))
        f.write(struct.pack('Q', data_size))
        f.write(output_data.flatten().astype(np.float32).tobytes())
    return bin_file

# Ana simülasyon ve çalıştırma fonksiyonu
def run_model(model_selection, layer_selection, data_choice, mode):
    model_info = MODELS[model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]

    # Modeli yükle
    try:
        loaded_model = tf.keras.models.load_model(model_path)
    except Exception as e:
        return f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Test verisini yükle
    try:
        if flow_from_directory:
            test_data = load_images_with_flow_from_directory(test_path, target_size=(224, 224), batch_size=8)
        else:
            test_data = load_images_from_directory(test_path, target_size=(224, 224), batch_size=8)
    except Exception as e:
        return f"Test verisi yükleme hatası: {str(e)}\nTest yolu: {test_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Katman seçimini işle
    layer_index = None
    current_layer = None
    if layer_selection and layer_selection != "None":
        try:
            layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
            current_layer = loaded_model.layers[layer_index]
        except (ValueError, IndexError) as e:
            return f"Katman seçimi hatası: {str(e)}\nSeçilen katman: {layer_selection}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Veri seçimini yap
    try:
        if data_choice == "Test Verisinden Örnek Kullan":
            if flow_from_directory:
                input_data = next(test_data)
            else:
                input_data = next(iter(test_data))
            data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
        else:
            expected_shape = loaded_model.input.shape[1:]
            input_data = np.random.random((8, *expected_shape))
            data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"
    except Exception as e:
        return f"Veri seçimi hatası: {str(e)}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    try:
        if layer_index is None or layer_selection == "None":
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            bin_file = None
        else:
            weights, window_params = extract_weights_and_window(current_layer)
            real_output = get_layer_output(loaded_model, layer_index, input_data)

            output_file_x = f"file_{uuid.uuid4()}.x"
            output_file_y = f"file_{uuid.uuid4()}.y"
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())
            with open(output_file_y, 'wb') as f:
                for w in weights:
                    f.write(w.flatten().tobytes())

            bin_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()), real_output)

            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=communicate_with_fpga,
                args=(real_output, output_file_x, result_queue, mode)
            )
            comm_thread.start()
            comm_thread.join()

            fpga_result = result_queue.get()
            if len(fpga_result) == 3:
                raise Exception(fpga_result[2])
            fpga_output, data_sent_size = fpga_result
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            os.remove(output_file_x)
            os.remove(output_file_y)
            if bin_file:
                os.remove(bin_file)

        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        output_str = f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Katman seçim dropdown'unu güncelleme fonksiyonu
def update_layer_dropdown(model_selection):
    try:
        model_info = MODELS[model_selection]
        model_path = model_info["model_path"]
        if not os.path.exists(model_path):
            return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                               info=f"Model dosyası bulunamadı: {model_path}")

        loaded_model = tf.keras.models.load_model(model_path)
        layer_names = ["None"]
        for i, layer in enumerate(loaded_model.layers):
            try:
                output_shape = layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: {output_shape}")
            except Exception as e:
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: Hata ({str(e)})")
        return gr.Dropdown(choices=layer_names, value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                           info=f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}")

# Gradio arayüzünü oluştur
with gr.Blocks(title="VGG-16 CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# VGG-16 CNN Model Hızlandırıcı")  # Başlık
    gr.Markdown("Modelinizi seçin, katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")  # Açıklama

    # Model ve mod seçimi
    with gr.Row():
        model_input = gr.Dropdown(choices=list(MODELS.keys()), label="Model Seçimi", value="VGG16-1")
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")

    # Katman seçimi ve simülasyon
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin (None = Hızlandırma Yok)")
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Model seçildiğinde katman dropdown'unu güncelle
    model_input.change(fn=update_layer_dropdown, inputs=model_input, outputs=layer_input)

    # Modeli çalıştır
    submit_btn.click(fn=run_model, inputs=[model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Arayüzü başlat
interface.launch()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a7a22db07931a2f2e5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# 3 model birleşimi

# Gerekli kütüphaneleri içe aktar
!pip install pyserial
!pip install gradio
import tensorflow as tf  # Sinir ağı modeli için
import numpy as np  # Sayısal işlemler için
from tensorflow.keras import layers  # Keras katmanları için
import gradio as gr  # Kullanıcı arayüzü için
import os  # Dosya işlemleri için
import threading  # Çoklu iş parçacığı için
import queue  # İş parçacıkları arası veri aktarımı için
import time  # Simülasyon gecikmesi için
import serial  # UART iletişimi için
import struct  # Binary veri paketleme için
import serial.tools.list_ports  # Mevcut seri portları listelemek için
import platform  # İşletim sistemi bilgisi için
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Resim yükleme ve işleme için
import uuid  # Benzersiz dosya adları için
from google.colab import drive  # Google Drive bağlantısı için

# Google Drive'ı bağla
drive.mount('/content/drive')

# Model ve veri yollarını tanımla
MODEL_TYPES = {
    "LeNet": {
        "models": {
            "LeNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (32, 32)
            },
            "LeNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (32, 32)
            },
            "LeNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (32, 32)
            }
        },
        "file_format": "h5"
    },
    "MobileNet": {
        "models": {
            "MobilNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet1/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "MobilNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet2/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "MobilNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet3/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    },
    "VGG-16": {
        "models": {
            "VGG16-1": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-lenetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "VGG16-2": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-vgg16sets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "VGG16-3": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-mobilnetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    }
}

# Veri yükleme fonksiyonu: Alt klasörsüz dizinlerden resimleri yükler
def load_images_from_directory(directory, target_size, batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

# Veri yükleme fonksiyonu: Alt klasörlü dizinlerden resimleri yükler
def load_images_with_flow_from_directory(directory, target_size, batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Ağırlık ve pencere parametrelerini çıkarma fonksiyonu
def extract_weights_and_window(layer):
    weights = layer.get_weights()
    window_params = {}
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = list(layer.kernel_size)
        window_params["stride"] = list(layer.strides)
        window_params["padding"] = layer.padding
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units
    return weights, window_params

# Mevcut seri portları listeleyen fonksiyon
def get_available_ports():
    ports = serial.tools.list_ports.comports()
    return [port.device for port in ports]

# FPGA ile iletişim fonksiyonu
def communicate_with_fpga(input_data, output_file, result_queue, mode):
    if mode == "Simülasyon":
        time.sleep(1)
        with open(output_file, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.float32)
        fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)
        result_queue.put((fpga_output, len(data)))
    else:
        try:
            default_port = "COM3" if platform.system() == "Windows" else "/dev/ttyUSB0"
            available_ports = get_available_ports()
            if default_port not in available_ports:
                raise Exception(f"Port {default_port} bulunamadı. Mevcut portlar: {available_ports or 'Yok'}")

            ser = serial.Serial(
                port=default_port,
                baudrate=115200,
                stopbits=serial.STOPBITS_TWO,
                bytesize=serial.EIGHTBITS,
                parity=serial.PARITY_NONE,
                timeout=5
            )

            with open(output_file, 'rb') as f:
                data = f.read()

            ser.write(data)
            ser.flush()

            response_size = len(data)
            response = ser.read(response_size)

            ser.close()

            if len(response) != response_size:
                raise Exception(f"FPGA'dan eksik veri alındı: {len(response)}/{response_size} bayt")

            fpga_output = np.frombuffer(response, dtype=np.float32).reshape(input_data.shape)
            result_queue.put((fpga_output, len(data)))

        except Exception as e:
            available_ports = get_available_ports()
            error_msg = f"FPGA iletişimi hatası: {str(e)}\nMevcut portlar: {available_ports or 'Yok'}\nLütfen doğru portu kontrol edin veya FPGA'nın bağlı olduğundan emin olun."
            result_queue.put((None, 0, error_msg))

# Katman çıktısını hesaplama fonksiyonu
def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Kağan için veriyi binary formatta kaydetme fonksiyonu
def save_data_for_kagan(weights, window_params, data_size, output_data):
    bin_file = f"data_for_kagan_{uuid.uuid4()}.bin"
    with open(bin_file, 'wb') as f:
        for w in weights:
            f.write(w.flatten().astype(np.float32).tobytes())
        if window_params:
            if "kernel_size" in window_params:
                f.write(struct.pack('ii', *window_params["kernel_size"]))
                f.write(struct.pack('ii', *window_params["stride"]))
                padding_val = 1 if window_params["padding"] == "valid" else 0
                f.write(struct.pack('i', padding_val))
            elif "units" in window_params:
                f.write(struct.pack('i', window_params["units"]))
        f.write(struct.pack('Q', data_size))
        f.write(output_data.flatten().astype(np.float32).tobytes())
    return bin_file

# Ana simülasyon ve çalıştırma fonksiyonu
def run_model(model_type, model_selection, layer_selection, data_choice, mode):
    model_info = MODEL_TYPES[model_type]["models"][model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]
    target_size = model_info["target_size"]
    file_format = MODEL_TYPES[model_type]["file_format"]

    # Modeli yükle
    try:
        loaded_model = tf.keras.models.load_model(model_path)
    except Exception as e:
        return f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Test verisini yükle
    try:
        if flow_from_directory:
            test_data = load_images_with_flow_from_directory(test_path, target_size=target_size, batch_size=8)
        else:
            test_data = load_images_from_directory(test_path, target_size=target_size, batch_size=8)
    except Exception as e:
        return f"Test verisi yükleme hatası: {str(e)}\nTest yolu: {test_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Katman seçimini işle
    layer_index = None
    current_layer = None
    if layer_selection and layer_selection != "None":
        try:
            layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
            current_layer = loaded_model.layers[layer_index]
        except (ValueError, IndexError) as e:
            return f"Katman seçimi hatası: {str(e)}\nSeçilen katman: {layer_selection}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Veri seçimini yap
    try:
        if data_choice == "Test Verisinden Örnek Kullan":
            if flow_from_directory:
                input_data = next(test_data)
            else:
                input_data = next(iter(test_data))
            data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
        else:
            expected_shape = loaded_model.input.shape[1:]
            input_data = np.random.random((8, *expected_shape))
            data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"
    except Exception as e:
        return f"Veri seçimi hatası: {str(e)}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    try:
        if layer_index is None or layer_selection == "None":
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            bin_file = None
        else:
            weights, window_params = extract_weights_and_window(current_layer)
            real_output = get_layer_output(loaded_model, layer_index, input_data)

            output_file_x = f"file_{uuid.uuid4()}.x"
            output_file_y = f"file_{uuid.uuid4()}.y"
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())
            with open(output_file_y, 'wb') as f:
                for w in weights:
                    f.write(w.flatten().tobytes())

            bin_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()), real_output)

            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=communicate_with_fpga,
                args=(real_output, output_file_x, result_queue, mode)
            )
            comm_thread.start()
            comm_thread.join()

            fpga_result = result_queue.get()
            if len(fpga_result) == 3:
                raise Exception(fpga_result[2])
            fpga_output, data_sent_size = fpga_result
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            os.remove(output_file_x)
            os.remove(output_file_y)
            if bin_file:
                os.remove(bin_file)

        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        output_str = f"Seçilen Model Türü: {model_type}\n"
        output_str += f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Model alt seçim dropdown'unu güncelleme fonksiyonu
def update_model_dropdown(model_type):
    try:
        models = list(MODEL_TYPES[model_type]["models"].keys())
        return gr.Dropdown(choices=models, value=models[0], label="Model Seçimi")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Model Seçimi", info=f"Hata: {str(e)}")

# Katman seçim dropdown'unu güncelleme fonksiyonu
def update_layer_dropdown(model_type, model_selection):
    try:
        model_info = MODEL_TYPES[model_type]["models"][model_selection]
        model_path = model_info["model_path"]
        if not os.path.exists(model_path):
            return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                               info=f"Model dosyası bulunamadı: {model_path}")

        loaded_model = tf.keras.models.load_model(model_path)
        layer_names = ["None"]
        for i, layer in enumerate(loaded_model.layers):
            try:
                output_shape = layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: {output_shape}")
            except Exception as e:
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: Hata ({str(e)})")
        return gr.Dropdown(choices=layer_names, value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                           info=f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}")

# Gradio arayüzünü oluştur
with gr.Blocks(title="CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# CNN Model Hızlandırıcı")
    gr.Markdown("Model türünüzü ve modelinizi seçin, ardından katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")

    # Model türü seçimi
    model_type_input = gr.Dropdown(choices=list(MODEL_TYPES.keys()), label="Model Türü Seçimi", value="LeNet")

    # Model ve mod seçimi
    with gr.Row():
        model_input = gr.Dropdown(label="Model Seçimi")
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")

    # Katman seçimi ve simülasyon
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin (None = Hızlandırma Yok)")
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Model türü değiştiğinde model dropdown'unu güncelle
    model_type_input.change(fn=update_model_dropdown, inputs=model_type_input, outputs=model_input)

    # Model veya model türü değiştiğinde katman dropdown'unu güncelle
    model_input.change(fn=update_layer_dropdown, inputs=[model_type_input, model_input], outputs=layer_input)
    model_type_input.change(fn=update_layer_dropdown, inputs=[model_type_input, model_input], outputs=layer_input)

    # Modeli çalıştır
    submit_btn.click(fn=run_model, inputs=[model_type_input, model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Arayüzü başlat
interface.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 8.5 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://39d4f853385d5a0d1b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# revize kod

# 3 model birleşimi

# Gerekli kütüphaneleri içe aktar
!pip install pyserial
!pip install gradio
import tensorflow as tf  # Sinir ağı modeli için
import numpy as np  # Sayısal işlemler için
from tensorflow.keras import layers  # Keras katmanları için
import gradio as gr  # Kullanıcı arayüzü için
import os  # Dosya işlemleri için
import threading  # Çoklu iş parçacığı için
import queue  # İş parçacıkları arası veri aktarımı için
import time  # Simülasyon gecikmesi için
import serial  # UART iletişimi için
import struct  # Binary veri paketleme için
import serial.tools.list_ports  # Mevcut seri portları listelemek için
import platform  # İşletim sistemi bilgisi için
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Resim yükleme ve işleme için
import uuid  # Benzersiz dosya adları için
from google.colab import drive  # Google Drive bağlantısı için

# Google Drive'ı bağla
drive.mount('/content/drive')

# Model ve veri yollarını tanımla
MODEL_TYPES = {
    "LeNet": {
        "models": {
            "LeNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (32, 32)
            },
            "LeNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (32, 32)
            },
            "LeNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (32, 32)
            }
        },
        "file_format": "h5"
    },
    "MobileNet": {
        "models": {
            "MobilNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet1/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "MobilNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet2/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "MobilNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet3/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    },
    "VGG-16": {
        "models": {
            "VGG16-1": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-lenetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "VGG16-2": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-vgg16sets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "VGG16-3": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-mobilnetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    }
}

# Veri yükleme fonksiyonu: Alt klasörsüz dizinlerden resimleri yükler
def load_images_from_directory(directory, target_size, batch_size=8):
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

# Veri yükleme fonksiyonu: Alt klasörlü dizinlerden resimleri yükler
def load_images_with_flow_from_directory(directory, target_size, batch_size=8):
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Ağırlık ve pencere parametrelerini çıkarma fonksiyonu
def extract_weights_and_window(layer):
    weights = layer.get_weights()
    window_params = {}
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = list(layer.kernel_size)
        window_params["stride"] = list(layer.strides)
        window_params["padding"] = layer.padding
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units
    return weights, window_params

# Mevcut seri portları listeleyen fonksiyon
def get_available_ports():
    ports = serial.tools.list_ports.comports()
    return [port.device for port in ports]

# FPGA ile iletişim fonksiyonu
def communicate_with_fpga(input_data, output_file, result_queue, mode):
    if mode == "Simülasyon":
        time.sleep(1)
        with open(output_file, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.float32)
        fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)
        result_queue.put((fpga_output, len(data)))
    else:
        try:
            default_port = "COM3" if platform.system() == "Windows" else "/dev/ttyUSB0"
            available_ports = get_available_ports()
            if default_port not in available_ports:
                raise Exception(f"Port {default_port} bulunamadı. Mevcut portlar: {available_ports or 'Yok'}")

            ser = serial.Serial(
                port=default_port,
                baudrate=115200,
                stopbits=serial.STOPBITS_TWO,
                bytesize=serial.EIGHTBITS,
                parity=serial.PARITY_NONE,
                timeout=5
            )

            with open(output_file, 'rb') as f:
                data = f.read()

            ser.write(data)
            ser.flush()

            response_size = len(data)
            response = ser.read(response_size)

            ser.close()

            if len(response) != response_size:
                raise Exception(f"FPGA'dan eksik veri alındı: {len(response)}/{response_size} bayt")

            fpga_output = np.frombuffer(response, dtype=np.float32).reshape(input_data.shape)
            result_queue.put((fpga_output, len(data)))

        except Exception as e:
            available_ports = get_available_ports()
            error_msg = f"FPGA iletişimi hatası: {str(e)}\nMevcut portlar: {available_ports or 'Yok'}\nLütfen doğru portu kontrol edin veya FPGA'nın bağlı olduğundan emin olun."
            result_queue.put((None, 0, error_msg))

# Katman çıktısını hesaplama fonksiyonu
def get_layer_output(model, layer_index, input_data):
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Kağan için veriyi binary formatta kaydetme fonksiyonu
def save_data_for_kagan(weights, window_params, data_size, output_data):
    bin_file = f"data_for_kagan_{uuid.uuid4()}.bin"
    with open(bin_file, 'wb') as f:
        for w in weights:
            f.write(w.flatten().astype(np.float32).tobytes())
        if window_params:
            if "kernel_size" in window_params:
                f.write(struct.pack('ii', *window_params["kernel_size"]))
                f.write(struct.pack('ii', *window_params["stride"]))
                padding_val = 1 if window_params["padding"] == "valid" else 0
                f.write(struct.pack('i', padding_val))
            elif "units" in window_params:
                f.write(struct.pack('i', window_params["units"]))
        f.write(struct.pack('Q', data_size))
        f.write(output_data.flatten().astype(np.float32).tobytes())
    return bin_file

# Ana simülasyon ve çalıştırma fonksiyonu
def run_model(model_type, model_selection, layer_selection, data_choice, mode):
    model_info = MODEL_TYPES[model_type]["models"][model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]
    target_size = model_info["target_size"]
    file_format = MODEL_TYPES[model_type]["file_format"]

    # Modeli yükle
    try:
        loaded_model = tf.keras.models.load_model(model_path)
    except Exception as e:
        return f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Test verisini yükle
    try:
        if flow_from_directory:
            test_data = load_images_with_flow_from_directory(test_path, target_size=target_size, batch_size=8)
        else:
            test_data = load_images_from_directory(test_path, target_size=target_size, batch_size=8)
    except Exception as e:
        return f"Test verisi yükleme hatası: {str(e)}\nTest yolu: {test_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Katman seçimini işle
    layer_index = None
    current_layer = None
    if mode == "Donanım" and layer_selection and layer_selection != "None":
        try:
            layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
            current_layer = loaded_model.layers[layer_index]
        except (ValueError, IndexError) as e:
            return f"Katman seçimi hatası: {str(e)}\nSeçilen katman: {layer_selection}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Veri seçimini yap
    try:
        if data_choice == "Test Verisinden Örnek Kullan":
            if flow_from_directory:
                input_data = next(test_data)
            else:
                input_data = next(iter(test_data))
            data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
        else:
            expected_shape = loaded_model.input.shape[1:]
            input_data = np.random.random((8, *expected_shape))
            data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"
    except Exception as e:
        return f"Veri seçimi hatası: {str(e)}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    try:
        if mode == "Simülasyon" or layer_selection == "None" or not layer_selection:
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            bin_file = None
        else:
            weights, window_params = extract_weights_and_window(current_layer)
            real_output = get_layer_output(loaded_model, layer_index, input_data)

            output_file_x = f"file_{uuid.uuid4()}.x"
            output_file_y = f"file_{uuid.uuid4()}.y"
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())
            with open(output_file_y, 'wb') as f:
                for w in weights:
                    f.write(w.flatten().tobytes())

            bin_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()), real_output)

            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=communicate_with_fpga,
                args=(real_output, output_file_x, result_queue, mode)
            )
            comm_thread.start()
            comm_thread.join()

            fpga_result = result_queue.get()
            if len(fpga_result) == 3:
                raise Exception(fpga_result[2])
            fpga_output, data_sent_size = fpga_result
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            os.remove(output_file_x)
            os.remove(output_file_y)
            if bin_file:
                os.remove(bin_file)

        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        output_str = f"Seçilen Model Türü: {model_type}\n"
        output_str += f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Model alt seçim dropdown'unu güncelleme fonksiyonu
def update_model_dropdown(model_type):
    try:
        models = list(MODEL_TYPES[model_type]["models"].keys())
        return gr.Dropdown(choices=models, value=models[0], label="Model Seçimi")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Model Seçimi", info=f"Hata: {str(e)}")

# Katman seçim dropdown'unu güncelleme fonksiyonu
def update_layer_dropdown(model_type, model_selection):
    try:
        model_info = MODEL_TYPES[model_type]["models"][model_selection]
        model_path = model_info["model_path"]
        if not os.path.exists(model_path):
            return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                               info=f"Model dosyası bulunamadı: {model_path}")

        loaded_model = tf.keras.models.load_model(model_path)
        layer_names = ["None"]
        for i, layer in enumerate(loaded_model.layers):
            try:
                output_shape = layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: {output_shape}")
            except Exception as e:
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: Hata ({str(e)})")
        return gr.Dropdown(choices=layer_names, value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                           info=f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}")

# Çalışma modu değiştiğinde katman dropdown'unun görünürlüğünü güncelleme
def update_layer_visibility(mode):
    return gr.update(visible=(mode == "Donanım"))

# Gradio arayüzünü oluştur
with gr.Blocks(title="CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# CNN Model Hızlandırıcı")
    gr.Markdown("Model türünüzü ve modelinizi seçin, ardından katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")

    # Model türü seçimi
    model_type_input = gr.Dropdown(choices=list(MODEL_TYPES.keys()), label="Model Türü Seçimi", value="LeNet")

    # Model ve mod seçimi
    with gr.Row():
        model_input = gr.Dropdown(label="Model Seçimi")
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")

    # Katman seçimi ve simülasyon
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin", visible=False)
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Model türü değiştiğinde model dropdown'unu güncelle
    model_type_input.change(fn=update_model_dropdown, inputs=model_type_input, outputs=model_input)

    # Model veya model türü değiştiğinde katman dropdown'unu güncelle
    model_input.change(fn=update_layer_dropdown, inputs=[model_type_input, model_input], outputs=layer_input)
    model_type_input.change(fn=update_layer_dropdown, inputs=[model_type_input, model_input], outputs=layer_input)

    # Çalışma modu değiştiğinde katman dropdown'unun görünürlüğünü güncelle
    mode_input.change(fn=update_layer_visibility, inputs=mode_input, outputs=layer_input)

    # Modeli çalıştır
    submit_btn.click(fn=run_model, inputs=[model_type_input, model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Arayüzü başlat
interface.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 8.9 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a7591dfec41c5f74bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Kod Açıklaması Part Part

# Bölüm 1: Kütüphane İçe Aktarmaları ve Kurulum
# Bu bölümde, projede kullanılan tüm kütüphaneler içe aktarılır ve Google Drive bağlantısı kurulur.
# Gerekli kütüphaneler: TensorFlow (sinir ağı), NumPy (sayısal işlemler), Gradio (kullanıcı arayüzü), PySerial (FPGA iletişimi) vb.

!pip install pyserial
!pip install gradio
import tensorflow as tf  # Sinir ağı modellerini oluşturmak ve çalıştırmak için
import numpy as np  # Sayısal işlemler ve dizi manipülasyonları için
from tensorflow.keras import layers  # Sinir ağı katmanlarını tanımlamak için
import gradio as gr  # İnteraktif kullanıcı arayüzü oluşturmak için
import os  # Dosya ve dizin işlemleri için
import threading  # FPGA ile paralel iletişim için çoklu iş parçacığı
import queue  # İş parçacıkları arasında veri aktarımı için kuyruk
import time  # Simülasyon modunda gecikme simülasyonu için
import serial  # FPGA ile seri port üzerinden iletişim için
import struct  # Binary veri paketleme ve açma için
import serial.tools.list_ports  # Mevcut seri portları listelemek için
import platform  # İşletim sistemi bilgisini almak için
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Resim yükleme ve ön işleme için
import uuid  # Benzersiz dosya adları oluşturmak için
from google.colab import drive  # Google Drive bağlantısı için

# Google Drive'ı bağla: Modeller ve veri setleri Google Drive'da saklanıyor
drive.mount('/content/drive')

# Bölüm 2: Model ve Veri Yolu Tanımlamaları
# Bu bölümde, LeNet, MobileNet ve VGG-16 modellerinin yolları ve veri seti bilgileri tanımlanır.
# Her model türü için farklı varyantlar ve test veri yolları belirtilir.

MODEL_TYPES = {
    "LeNet": {
        "models": {
            "LeNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (32, 32)
            },
            "LeNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (32, 32)
            },
            "LeNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (32, 32)
            }
        },
        "file_format": "h5"
    },
    "MobileNet": {
        "models": {
            "MobilNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet1/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "MobilNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet2/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "MobilNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet3/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    },
    "VGG-16": {
        "models": {
            "VGG16-1": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-lenetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "VGG16-2": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-vgg16sets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "VGG16-3": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-mobilnetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    }
}

# Bölüm 3: Veri Yükleme Fonksiyonları
# Bu bölümde, test verilerini yüklemek için iki fonksiyon tanımlanır:
# - load_images_from_directory: Alt klasörsüz dizinlerden resimleri yükler
# - load_images_with_flow_from_directory: Alt klasörlü dizinlerden resimleri yükler

def load_images_from_directory(directory, target_size, batch_size=8):
    # Alt klasörsüz dizinlerden resimleri yükler ve ön işleme yapar
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0  # Normalizasyon
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

def load_images_with_flow_from_directory(directory, target_size, batch_size=8):
    # Alt klasörlü dizinlerden resimleri yükler ve veri artırma uygular
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Bölüm 4: Ağırlık ve Pencere Parametreleri Çıkarma
# Bu bölümde, seçilen katmanın ağırlıklarını ve pencere parametrelerini (ör. kernel boyutu, adım) çıkaran bir fonksiyon tanımlanır.

def extract_weights_and_window(layer):
    # Katmanın ağırlıklarını ve pencere parametrelerini (Conv2D veya Dense katmanları için) çıkarır
    weights = layer.get_weights()
    window_params = {}
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = list(layer.kernel_size)
        window_params["stride"] = list(layer.strides)
        window_params["padding"] = layer.padding
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units
    return weights, window_params

# Bölüm 5: Seri Port ve FPGA İletişimi
# Bu bölümde, seri portları listelemek ve FPGA ile iletişim kurmak için fonksiyonlar tanımlanır.

def get_available_ports():
    # Mevcut seri portları listeler
    ports = serial.tools.list_ports.comports()
    return [port.device for port in ports]

def communicate_with_fpga(input_data, output_file, result_queue, mode):
    # FPGA ile iletişim kurar: Simülasyon modunda simüle veri üretir, Donanım modunda seri port üzerinden veri gönderir
    if mode == "Simülasyon":
        time.sleep(1)  # Simülasyon için gecikme
        with open(output_file, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.float32)
        fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)  # Gürültü ekler
        result_queue.put((fpga_output, len(data)))
    else:
        try:
            default_port = "COM3" if platform.system() == "Windows" else "/dev/ttyUSB0"
            available_ports = get_available_ports()
            if default_port not in available_ports:
                raise Exception(f"Port {default_port} bulunamadı. Mevcut portlar: {available_ports or 'Yok'}")

            ser = serial.Serial(
                port=default_port,
                baudrate=115200,
                stopbits=serial.STOPBITS_TWO,
                bytesize=serial.EIGHTBITS,
                parity=serial.PARITY_NONE,
                timeout=5
            )

            with open(output_file, 'rb') as f:
                data = f.read()

            ser.write(data)
            ser.flush()

            response_size = len(data)
            response = ser.read(response_size)

            ser.close()

            if len(response) != response_size:
                raise Exception(f"FPGA'dan eksik veri alındı: {len(response)}/{response_size} bayt")

            fpga_output = np.frombuffer(response, dtype=np.float32).reshape(input_data.shape)
            result_queue.put((fpga_output, len(data)))

        except Exception as e:
            available_ports = get_available_ports()
            error_msg = f"FPGA iletişimi hatası: {str(e)}\nMevcut portlar: {available_ports or 'Yok'}\nLütfen doğru portu kontrol edin veya FPGA'nın bağlı olduğundan emin olun."
            result_queue.put((None, 0, error_msg))

# Bölüm 6: Katman Çıktısı Hesaplama
# Bu bölümde, seçilen katmanın çıktısını hesaplayan bir fonksiyon tanımlanır.

def get_layer_output(model, layer_index, input_data):
    # Belirtilen katmanın çıktısını hesaplar
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Bölüm 7: Kağan için Veri Hazırlama
# Bu bölümde, ağırlıklar ve çıktılar binary formatta kaydedilir (FPGA için veri hazırlığı).

def save_data_for_kagan(weights, window_params, data_size, output_data):
    # Ağırlıkları, pencere parametrelerini ve çıktıları binary dosyaya kaydeder
    bin_file = f"data_for_kagan_{uuid.uuid4()}.bin"
    with open(bin_file, 'wb') as f:
        for w in weights:
            f.write(w.flatten().astype(np.float32).tobytes())
        if window_params:
            if "kernel_size" in window_params:
                f.write(struct.pack('ii', *window_params["kernel_size"]))
                f.write(struct.pack('ii', *window_params["stride"]))
                padding_val = 1 if window_params["padding"] == "valid" else 0
                f.write(struct.pack('i', padding_val))
            elif "units" in window_params:
                f.write(struct.pack('i', window_params["units"]))
        f.write(struct.pack('Q', data_size))
        f.write(output_data.flatten().astype(np.float32).tobytes())
    return bin_file

# Bölüm 8: Ana Simülasyon ve Çalıştırma Fonksiyonu
# Bu bölümde, modelin çalıştırılması, FPGA ile iletişim ve sonuçların karşılaştırılması gerçekleştirilir.

def run_model(model_type, model_selection, layer_selection, data_choice, mode):
    # Seçilen model, katman ve mod ile modeli çalıştırır ve FPGA çıktısını gerçek çıktı ile karşılaştırır
    model_info = MODEL_TYPES[model_type]["models"][model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]
    target_size = model_info["target_size"]
    file_format = MODEL_TYPES[model_type]["file_format"]

    # Modeli yükle
    try:
        loaded_model = tf.keras.models.load_model(model_path)
    except Exception as e:
        return f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Test verisini yükle
    try:
        if flow_from_directory:
            test_data = load_images_with_flow_from_directory(test_path, target_size=target_size, batch_size=8)
        else:
            test_data = load_images_from_directory(test_path, target_size=target_size, batch_size=8)
    except Exception as e:
        return f"Test verisi yükleme hatası: {str(e)}\nTest yolu: {test_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Katman seçimini işle (sadece Donanım modunda)
    layer_index = None
    current_layer = None
    if mode == "Donanım" and layer_selection and layer_selection != "None":
        try:
            layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
            current_layer = loaded_model.layers[layer_index]
        except (ValueError, IndexError) as e:
            return f"Katman seçimi hatası: {str(e)}\nSeçilen katman: {layer_selection}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Veri seçimini yap
    try:
        if data_choice == "Test Verisinden Örnek Kullan":
            if flow_from_directory:
                input_data = next(test_data)
            else:
                input_data = next(iter(test_data))
            data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
        else:
            expected_shape = loaded_model.input.shape[1:]
            input_data = np.random.random((8, *expected_shape))
            data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"
    except Exception as e:
        return f"Veri seçimi hatası: {str(e)}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    try:
        if mode == "Simülasyon" or layer_selection == "None" or not layer_selection:
            # Simülasyon modunda veya katman seçilmediğinde tam modeli çalıştır
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            bin_file = None
        else:
            # Donanım modunda seçilen katmanın çıktısını hesapla ve FPGA ile iletişim kur
            weights, window_params = extract_weights_and_window(current_layer)
            real_output = get_layer_output(loaded_model, layer_index, input_data)

            output_file_x = f"file_{uuid.uuid4()}.x"
            output_file_y = f"file_{uuid.uuid4()}.y"
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())
            with open(output_file_y, 'wb') as f:
                for w in weights:
                    f.write(w.flatten().tobytes())

            bin_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()), real_output)

            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=communicate_with_fpga,
                args=(real_output, output_file_x, result_queue, mode)
            )
            comm_thread.start()
            comm_thread.join()

            fpga_result = result_queue.get()
            if len(fpga_result) == 3:
                raise Exception(fpga_result[2])
            fpga_output, data_sent_size = fpga_result
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            os.remove(output_file_x)
            os.remove(output_file_y)
            if bin_file:
                os.remove(bin_file)

        # Çıktıları karşılaştır ve benzerlik skorunu hesapla
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        # Sonuçları formatla
        output_str = f"Seçilen Model Türü: {model_type}\n"
        output_str += f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        # Benzerlik skorunu görselleştir
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Bölüm 9: Gradio Arayüz Yardımcı Fonksiyonları
# Bu bölümde, Gradio arayüzünün dinamik bileşenlerini güncelleyen fonksiyonlar tanımlanır.

def update_model_dropdown(model_type):
    # Model türü seçildiğinde model seçim dropdown'unu günceller
    try:
        models = list(MODEL_TYPES[model_type]["models"].keys())
        return gr.Dropdown(choices=models, value=models[0], label="Model Seçimi")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Model Seçimi", info=f"Hata: {str(e)}")

def update_layer_dropdown(model_type, model_selection):
    # Model veya model türü değiştiğinde katman seçim dropdown'unu günceller
    try:
        model_info = MODEL_TYPES[model_type]["models"][model_selection]
        model_path = model_info["model_path"]
        if not os.path.exists(model_path):
            return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                               info=f"Model dosyası bulunamadı: {model_path}")

        loaded_model = tf.keras.models.load_model(model_path)
        layer_names = ["None"]
        for i, layer in enumerate(loaded_model.layers):
            try:
                output_shape = layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: {output_shape}")
            except Exception as e:
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: Hata ({str(e)})")
        return gr.Dropdown(choices=layer_names, value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                           info=f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}")

def update_layer_visibility(mode):
    # Çalışma modu değiştiğinde katman seçim dropdown'unun görünürlüğünü kontrol eder
    return gr.update(visible=(mode == "Donanım"))

# Bölüm 10: Gradio Kullanıcı Arayüzü
# Bu bölümde, interaktif Gradio arayüzü oluşturulur ve kullanıcı girişleri alınır.

with gr.Blocks(title="CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# CNN Model Hızlandırıcı")
    gr.Markdown("Model türünüzü ve modelinizi seçin, ardından katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")

    # Model türü seçimi
    model_type_input = gr.Dropdown(choices=list(MODEL_TYPES.keys()), label="Model Türü Seçimi", value="LeNet")

    # Model ve mod seçimi
    with gr.Row():
        model_input = gr.Dropdown(label="Model Seçimi")
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")

    # Katman seçimi ve simülasyon
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin", visible=False)
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Model türü değiştiğinde model dropdown'unu güncelle
    model_type_input.change(fn=update_model_dropdown, inputs=model_type_input, outputs=model_input)

    # Model veya model türü değiştiğinde katman dropdown'unu güncelle
    model_input.change(fn=update_layer_dropdown, inputs=[model_type_input, model_input], outputs=layer_input)
    model_type_input.change(fn=update_layer_dropdown, inputs=[model_type_input, model_input], outputs=layer_input)

    # Çalışma modu değiştiğinde katman dropdown'unun görünürlüğünü güncelle
    mode_input.change(fn=update_layer_visibility, inputs=mode_input, outputs=layer_input)

    # Modeli çalıştır
    submit_btn.click(fn=run_model, inputs=[model_type_input, model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Arayüzü başlat
interface.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.3 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1651567a529ef5338f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#26.08.2025 Grubun ortak phyton kodu deneme1


# Kod Açıklaması Part Part

# Bölüm 1: Kütüphane İçe Aktarmaları ve Kurulum
# Bu bölümde, projede kullanılan tüm kütüphaneler içe aktarılır ve Google Drive bağlantısı kurulur.
# Gerekli kütüphaneler: TensorFlow (sinir ağı), NumPy (sayısal işlemler), Gradio (kullanıcı arayüzü), PySerial (FPGA iletişimi) vb.

!pip install pyserial
!pip install gradio
import tensorflow as tf  # Sinir ağı modellerini oluşturmak ve çalıştırmak için
import numpy as np  # Sayısal işlemler ve dizi manipülasyonları için
from tensorflow.keras import layers  # Sinir ağı katmanlarını tanımlamak için
import gradio as gr  # İnteraktif kullanıcı arayüzü oluşturmak için
import os  # Dosya ve dizin işlemleri için
import threading  # FPGA ile paralel iletişim için çoklu iş parçacığı
import queue  # İş parçacıkları arasında veri aktarımı için kuyruk
import time  # Simülasyon modunda gecikme simülasyonu için
import serial  # FPGA ile seri port üzerinden iletişim için
import struct  # Binary veri paketleme ve açma için
import serial.tools.list_ports  # Mevcut seri portları listelemek için
import platform  # İşletim sistemi bilgisini almak için
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Resim yükleme ve ön işleme için
import uuid  # Benzersiz dosya adları oluşturmak için
from google.colab import drive  # Google Drive bağlantısı için

# Google Drive'ı bağla: Modeller ve veri setleri Google Drive'da saklanıyor
drive.mount('/content/drive')

# Bölüm 2: Model ve Veri Yolu Tanımlamaları
# Bu bölümde, LeNet, MobileNet ve VGG-16 modellerinin yolları ve veri seti bilgileri tanımlanır.
# Her model türü için farklı varyantlar ve test veri yolları belirtilir.

MODEL_TYPES = {
    "LeNet": {
        "models": {
            "LeNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (32, 32)
            },
            "LeNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (32, 32)
            },
            "LeNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (32, 32)
            }
        },
        "file_format": "h5"
    },
    "MobileNet": {
        "models": {
            "MobilNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet1/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "MobilNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet2/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "MobilNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet3/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    },
    "VGG-16": {
        "models": {
            "VGG16-1": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-lenetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "VGG16-2": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-vgg16sets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "VGG16-3": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-mobilnetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    }
}

# Bölüm 3: Veri Yükleme Fonksiyonları
# Bu bölümde, test verilerini yüklemek için iki fonksiyon tanımlanır:
# - load_images_from_directory: Alt klasörsüz dizinlerden resimleri yükler
# - load_images_with_flow_from_directory: Alt klasörlü dizinlerden resimleri yükler

def load_images_from_directory(directory, target_size, batch_size=8):
    # Alt klasörsüz dizinlerden resimleri yükler ve ön işleme yapar
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0  # Normalizasyon
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

def load_images_with_flow_from_directory(directory, target_size, batch_size=8):
    # Alt klasörlü dizinlerden resimleri yükler ve veri artırma uygular
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Bölüm 4: Ağırlık ve Pencere Parametreleri Çıkarma
# Bu bölümde, seçilen katmanın ağırlıklarını ve pencere parametrelerini (ör. kernel boyutu, adım) çıkaran bir fonksiyon tanımlanır.

def extract_weights_and_window(layer):
    # Katmanın ağırlıklarını ve pencere parametrelerini (Conv2D veya Dense katmanları için) çıkarır
    weights = layer.get_weights()
    window_params = {}
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = list(layer.kernel_size)
        window_params["stride"] = list(layer.strides)
        window_params["padding"] = layer.padding
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units
    return weights, window_params

# Bölüm 5: Seri Port ve FPGA İletişimi
# Bu bölümde, seri portları listelemek ve FPGA ile iletişim kurmak için fonksiyonlar tanımlanır.

def get_available_ports():
    # Mevcut seri portları listeler
    ports = serial.tools.list_ports.comports()
    return [port.device for port in ports]

# FPGA kodundan entegre edilen fonksiyonlar
def float2hex(data: float) -> int:
    ret = int()
    ret = struct.unpack('!I', struct.pack('!f', data))[0]
    return ret

def to_four_bytes_little(data) -> bytearray:
    to4 = struct.Struct('<I').pack
    dummy = float2hex(data)
    ret = bytearray(to4(dummy & 0xFFFFFFFF))
    return ret

def array2bytearray_little(data: np.ndarray, dim: int) -> bytearray:
    ret = bytearray()
    if dim == 1 :
        for i in range(0, data.shape[0]):
            ret.extend(to_four_bytes_little(data[i]))
        return ret
    elif dim == 2:
        for i in range(0, data.shape[0]):
            for j in range(0, data.shape[1]):
                ret.extend(to_four_bytes_little(data[i,j]))
        return ret
    else:
        return ret

def byt_conv(data: int) -> bytes:
    ret = data.to_bytes(4,'little')
    return ret

def intarray2bytearray(data: np.ndarray) -> bytearray:
    ret = bytearray()
    for i in range(0, len(data)):
        ret.extend(byt_conv(data[i]))
    return ret

def read_float(seri: serial) -> float:
    dat = bytes()
    dat = seri.read(4)
    dum = struct.unpack('<I', dat)[0]
    dat = struct.pack('>I', dum)
    datflo = struct.unpack('>f', dat)
    return datflo[0]

def recv_res(seri: serial, row, col):
    res = np.zeros((row,col), dtype=float)
    for i in range(0,row):
        for j in range(0,col):
            res[i,j] = read_float(seri)
    return res

def Conv_HW(seri: serial, data_rows: int, data_cols: int, kernel_size: int, data_matrice, kernel):
    result_rows = data_rows - kernel_size + 1
    result_cols = data_cols - kernel_size + 1
    ram1_size: int = kernel_size * kernel_size * 4
    ram2_size: int = data_rows * data_cols * 4
    ram3_size: int = result_cols * result_cols * 4
    params = [data_rows, data_cols, kernel_size, result_rows, ram1_size, ram2_size, ram3_size, result_cols]
    buf1 = bytearray()
    for i in range(0, len(params)):
        buf1.extend(byt_conv(params[i]))
    buf2 = array2bytearray_little(kernel, 2)
    buf3 = array2bytearray_little(data_matrice, 2)
    seri.write(buf1)
    seri.write(buf2)
    seri.write(buf3)
    result = recv_res(seri, result_rows, result_cols)
    return result

def communicate_with_fpga(input_data, output_file, result_queue, mode):

    if mode == "Simülasyon":
        time.sleep(1)
        with open(output_file, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.float32)
        fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)
        result_queue.put((fpga_output, len(data)))
    else:
        try:
            ser = serial.Serial(
                port='COM1',
                baudrate=115200,
                stopbits=serial.STOPBITS_TWO,
                parity=serial.PARITY_NONE,
                timeout=2
            )

            # Conv_HW'yi kullanarak hesaplama yap
            # Not: input_data'nın 2D olduğu varsayılıyor (örneğin tek kanal için). Çok kanallı için uyarlama gerekebilir.
            # Burada basitlik için input_data'nın (rows, cols) şeklinde olduğunu varsayıyoruz; gerekirse reshape edin.
            if len(input_data.shape) > 2:
                # Örnek: Tek batch ve kanal için flatten to 2D; gerçekte loop ile kanalları işleyin.
                data_matrice = input_data[0, :, :, 0]  # İlk batch, ilk kanal örneği; uyarlayın.
            else:
                data_matrice = input_data

            # Kernel de benzer şekilde; weights[0] kernel (height, width, in_channels, out_channels)
            # Basitlik için tek kanal kernel alıyoruz; tam entegrasyon için loop ekleyin.
            kernel = weights[0][:, :, 0, 0]  # Örnek: İlk filtre; gerçekte tüm filtreleri işleyin.

            data_rows, data_cols = data_matrice.shape
            kernel_size = kernel.shape[0]  # Kare kernel varsayarak

            fpga_output = Conv_HW(ser, data_rows, data_cols, kernel_size, data_matrice, kernel)

            # fpga_output'ı orijinal shape'e uyarla
            fpga_output = np.expand_dims(fpga_output, axis=(0, -1))  # Örnek uyarlama; shape'i input_data'ya göre ayarlayın.

            data_sent_size = (data_rows * data_cols + kernel_size * kernel_size) * 4  # Yaklaşık boyut

            ser.close()

            result_queue.put((fpga_output, data_sent_size))

        except Exception as e:
            available_ports = get_available_ports()
            error_msg = f"FPGA iletişimi hatası: {str(e)}\nMevcut portlar: {available_ports or 'Yok'}\nLütfen doğru portu kontrol edin veya FPGA'nın bağlı olduğundan emin olun."
            result_queue.put((None, 0, error_msg))

# Bölüm 6: Katman Çıktısı Hesaplama
# Bu bölümde, seçilen katmanın çıktısını hesaplayan bir fonksiyon tanımlanır.

def get_layer_output(model, layer_index, input_data):
    # Belirtilen katmanın çıktısını hesaplar
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Bölüm 7: Kağan için Veri Hazırlama
# Bu bölümde, ağırlıklar ve çıktılar binary formatta kaydedilir (FPGA için veri hazırlığı).

def save_data_for_kagan(weights, window_params, data_size, output_data):
    # Ağırlıkları, pencere parametrelerini ve çıktıları binary dosyaya kaydeder
    bin_file = f"data_for_kagan_{uuid.uuid4()}.bin"
    with open(bin_file, 'wb') as f:
        for w in weights:
            f.write(w.flatten().astype(np.float32).tobytes())
        if window_params:
            if "kernel_size" in window_params:
                f.write(struct.pack('ii', *window_params["kernel_size"]))
                f.write(struct.pack('ii', *window_params["stride"]))
                padding_val = 1 if window_params["padding"] == "valid" else 0
                f.write(struct.pack('i', padding_val))
            elif "units" in window_params:
                f.write(struct.pack('i', window_params["units"]))
        f.write(struct.pack('Q', data_size))
        f.write(output_data.flatten().astype(np.float32).tobytes())
    return bin_file

# Bölüm 8: Ana Simülasyon ve Çalıştırma Fonksiyonu
# Bu bölümde, modelin çalıştırılması, FPGA ile iletişim ve sonuçların karşılaştırılması gerçekleştirilir.

def run_model(model_type, model_selection, layer_selection, data_choice, mode):
    # Seçilen model, katman ve mod ile modeli çalıştırır ve FPGA çıktısını gerçek çıktı ile karşılaştırır
    model_info = MODEL_TYPES[model_type]["models"][model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]
    target_size = model_info["target_size"]
    file_format = MODEL_TYPES[model_type]["file_format"]

    # Modeli yükle
    try:
        loaded_model = tf.keras.models.load_model(model_path)
    except Exception as e:
        return f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Test verisini yükle
    try:
        if flow_from_directory:
            test_data = load_images_with_flow_from_directory(test_path, target_size=target_size, batch_size=8)
        else:
            test_data = load_images_from_directory(test_path, target_size=target_size, batch_size=8)
    except Exception as e:
        return f"Test verisi yükleme hatası: {str(e)}\nTest yolu: {test_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Katman seçimini işle (sadece Donanım modunda)
    layer_index = None
    current_layer = None
    if mode == "Donanım" and layer_selection and layer_selection != "None":
        try:
            layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
            current_layer = loaded_model.layers[layer_index]
        except (ValueError, IndexError) as e:
            return f"Katman seçimi hatası: {str(e)}\nSeçilen katman: {layer_selection}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Veri seçimini yap
    try:
        if data_choice == "Test Verisinden Örnek Kullan":
            if flow_from_directory:
                input_data = next(test_data)
            else:
                input_data = next(iter(test_data))
            data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
        else:
            expected_shape = loaded_model.input.shape[1:]
            input_data = np.random.random((8, *expected_shape))
            data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"
    except Exception as e:
        return f"Veri seçimi hatası: {str(e)}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    try:
        if mode == "Simülasyon" or layer_selection == "None" or not layer_selection:
            # Simülasyon modunda veya katman seçilmediğinde tam modeli çalıştır
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            bin_file = None
        else:
            # Donanım modunda seçilen katmanın çıktısını hesapla ve FPGA ile iletişim kur
            global weights  # Conv_HW için weights'i global yap (entegre ederken erişim için)
            weights, window_params = extract_weights_and_window(current_layer)
            real_output = get_layer_output(loaded_model, layer_index, input_data)

            output_file_x = f"file_{uuid.uuid4()}.x"
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())  # input_data yerine real_output (katman inputu) kullan; ama Conv_HW için data_matrice katman inputu olmalı.

            bin_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()), real_output)

            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=communicate_with_fpga,
                args=(real_output, output_file_x, result_queue, mode)  # input_data yerine katman inputu (real_output öncesi output)
            )
            comm_thread.start()
            comm_thread.join()

            fpga_result = result_queue.get()
            if len(fpga_result) == 3:
                raise Exception(fpga_result[2])
            fpga_output, data_sent_size = fpga_result
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            os.remove(output_file_x)
            if bin_file:
                os.remove(bin_file)

        # Çıktıları karşılaştır ve benzerlik skorunu hesapla
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        # Sonuçları formatla
        output_str = f"Seçilen Model Türü: {model_type}\n"
        output_str += f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        # Benzerlik skorunu görselleştir
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Bölüm 9: Gradio Arayüz Yardımcı Fonksiyonları
# Bu bölümde, Gradio arayüzünün dinamik bileşenlerini güncelleyen fonksiyonlar tanımlanır.

def update_model_dropdown(model_type):
    # Model türü seçildiğinde model seçim dropdown'unu günceller
    try:
        models = list(MODEL_TYPES[model_type]["models"].keys())
        return gr.Dropdown(choices=models, value=models[0], label="Model Seçimi")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Model Seçimi", info=f"Hata: {str(e)}")

def update_layer_dropdown(model_type, model_selection):
    # Model veya model türü değiştiğinde katman seçim dropdown'unu günceller
    try:
        model_info = MODEL_TYPES[model_type]["models"][model_selection]
        model_path = model_info["model_path"]
        if not os.path.exists(model_path):
            return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                               info=f"Model dosyası bulunamadı: {model_path}")

        loaded_model = tf.keras.models.load_model(model_path)
        layer_names = ["None"]
        for i, layer in enumerate(loaded_model.layers):
            try:
                output_shape = layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: {output_shape}")
            except Exception as e:
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: Hata ({str(e)})")
        return gr.Dropdown(choices=layer_names, value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                           info=f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}")

def update_layer_visibility(mode):
    # Çalışma modu değiştiğinde katman seçim dropdown'unun görünürlüğünü kontrol eder
    return gr.update(visible=(mode == "Donanım"))

# Bölüm 10: Gradio Kullanıcı Arayüzü
# Bu bölümde, interaktif Gradio arayüzü oluşturulur ve kullanıcı girişleri alınır.

with gr.Blocks(title="CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# CNN Model Hızlandırıcı")
    gr.Markdown("Model türünüzü ve modelinizi seçin, ardından katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")

    # Model türü seçimi
    model_type_input = gr.Dropdown(choices=list(MODEL_TYPES.keys()), label="Model Türü Seçimi", value="LeNet")

    # Model ve mod seçimi
    with gr.Row():
        model_input = gr.Dropdown(label="Model Seçimi")
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")

    # Katman seçimi ve simülasyon
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin", visible=False)
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Model türü değiştiğinde model dropdown'unu güncelle
    model_type_input.change(fn=update_model_dropdown, inputs=model_type_input, outputs=model_input)

    # Model veya model türü değiştiğinde katman dropdown'unu güncelle
    model_input.change(fn=update_layer_dropdown, inputs=[model_type_input, model_input], outputs=layer_input)
    model_type_input.change(fn=update_layer_dropdown, inputs=[model_type_input, model_input], outputs=layer_input)

    # Çalışma modu değiştiğinde katman dropdown'unun görünürlüğünü güncelle
    mode_input.change(fn=update_layer_visibility, inputs=mode_input, outputs=layer_input)

    # Modeli çalıştır
    submit_btn.click(fn=run_model, inputs=[model_type_input, model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Arayüzü başlat
interface.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 10.5 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://40b5f1b9c124330031.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#yeni güncel
# 26.08.2025 Grubun ortak Python kodu deneme1

# Kod Açıklaması Part Part

# Bölüm 1: Kütüphane İçe Aktarmaları ve Kurulum
# Bu bölümde, projede kullanılan tüm kütüphaneler içe aktarılır ve Google Drive bağlantısı kurulur.
# Gerekli kütüphaneler: TensorFlow (sinir ağı), NumPy (sayısal işlemler), Gradio (kullanıcı arayüzü), PySerial (FPGA iletişimi) vb.

!pip install pyserial
!pip install gradio
import tensorflow as tf  # Sinir ağı modellerini oluşturmak ve çalıştırmak için
import numpy as np  # Sayısal işlemler ve dizi manipülasyonları için
from tensorflow.keras import layers  # Sinir ağı katmanlarını tanımlamak için
import gradio as gr  # İnteraktif kullanıcı arayüzü oluşturmak için
import os  # Dosya ve dizin işlemleri için
import threading  # FPGA ile paralel iletişim için çoklu iş parçacığı
import queue  # İş parçacıkları arasında veri aktarımı için kuyruk
import time  # Simülasyon modunda gecikme simülasyonu için
import serial  # FPGA ile seri port üzerinden iletişim için
import struct  # Binary veri paketleme ve açma için
import serial.tools.list_ports  # Mevcut seri portları listelemek için
import platform  # İşletim sistemi bilgisini almak için
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Resim yükleme ve ön işleme için
import uuid  # Benzersiz dosya adları oluşturmak için
from google.colab import drive  # Google Drive bağlantısı için

# Google Drive'ı bağla: Modeller ve veri setleri Google Drive'da saklanıyor
drive.mount('/content/drive')

# Bölüm 2: Model ve Veri Yolu Tanımlamaları
# Bu bölümde, LeNet, MobileNet ve VGG-16 modellerinin yolları ve veri seti bilgileri tanımlanır.
# Her model türü için farklı varyantlar ve test veri yolları belirtilir.

MODEL_TYPES = {
    "LeNet": {
        "models": {
            "LeNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (32, 32)
            },
            "LeNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (32, 32)
            },
            "LeNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (32, 32)
            }
        },
        "file_format": "h5"
    },
    "MobileNet": {
        "models": {
            "MobilNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet1/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "MobilNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet2/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "MobilNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet3/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    },
    "VGG-16": {
        "models": {
            "VGG16-1": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-lenetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "VGG16-2": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-vgg16sets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "VGG16-3": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-mobilnetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    }
}

# Bölüm 3: Veri Yükleme Fonksiyonları
# Bu bölümde, test verilerini yüklemek için iki fonksiyon tanımlanır:
# - load_images_from_directory: Alt klasörsüz dizinlerden resimleri yükler
# - load_images_with_flow_from_directory: Alt klasörlü dizinlerden resimleri yükler

def load_images_from_directory(directory, target_size, batch_size=8):
    # Alt klasörsüz dizinlerden resimleri yükler ve ön işleme yapar
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0  # Normalizasyon
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

def load_images_with_flow_from_directory(directory, target_size, batch_size=8):
    # Alt klasörlü dizinlerden resimleri yükler ve veri artırma uygular
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Bölüm 4: Ağırlık ve Pencere Parametreleri Çıkarma
# Bu bölümde, seçilen katmanın ağırlıklarını ve pencere parametrelerini (ör. kernel boyutu, adım) çıkaran bir fonksiyon tanımlanır.

def extract_weights_and_window(layer):
    # Katmanın ağırlıklarını ve pencere parametrelerini (Conv2D veya Dense katmanları için) çıkarır
    weights = layer.get_weights()
    window_params = {}
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = list(layer.kernel_size)
        window_params["stride"] = list(layer.strides)
        window_params["padding"] = layer.padding
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units
    return weights, window_params

# Bölüm 5: Seri Port ve FPGA İletişimi
# Bu bölümde, seri portları listelemek ve FPGA ile iletişim kurmak için fonksiyonlar tanımlanır.

def get_available_ports():
    # Mevcut seri portları listeler
    ports = serial.tools.list_ports.comports()
    return [port.device for port in ports]

# FPGA kodundan entegre edilen fonksiyonlar
def float2hex(data: float) -> int:
    ret = int()
    ret = struct.unpack('!I', struct.pack('!f', data))[0]
    return ret

def to_four_bytes_little(data) -> bytearray:
    to4 = struct.Struct('<I').pack
    dummy = float2hex(data)
    ret = bytearray(to4(dummy & 0xFFFFFFFF))
    return ret

def array2bytearray_little(data: np.ndarray, dim: int) -> bytearray:
    ret = bytearray()
    if dim == 1:
        for i in range(0, data.shape[0]):
            ret.extend(to_four_bytes_little(data[i]))
        return ret
    elif dim == 2:
        for i in range(0, data.shape[0]):
            for j in range(0, data.shape[1]):
                ret.extend(to_four_bytes_little(data[i,j]))
        return ret
    else:
        return ret

def byt_conv(data: int) -> bytes:
    ret = data.to_bytes(4,'little')
    return ret

def intarray2bytearray(data: np.ndarray) -> bytearray:
    ret = bytearray()
    for i in range(0, len(data)):
        ret.extend(byt_conv(data[i]))
    return ret

def read_float(seri: serial) -> float:
    dat = bytes()
    dat = seri.read(4)
    dum = struct.unpack('<I', dat)[0]
    dat = struct.pack('>I', dum)
    datflo = struct.unpack('>f', dat)
    return datflo[0]

def recv_res(seri: serial, row, col):
    res = np.zeros((row,col), dtype=float)
    for i in range(0,row):
        for j in range(0,col):
            res[i,j] = read_float(seri)
    return res

def Conv_HW(seri: serial, data_rows: int, data_cols: int, kernel_size: int, data_matrice, kernel):
    result_rows = data_rows - kernel_size + 1
    result_cols = data_cols - kernel_size + 1
    ram1_size: int = kernel_size * kernel_size * 4
    ram2_size: int = data_rows * data_cols * 4
    ram3_size: int = result_cols * result_cols * 4
    params = [data_rows, data_cols, kernel_size, result_rows, ram1_size, ram2_size, ram3_size, result_cols]
    buf1 = bytearray()
    for i in range(0, len(params)):
        buf1.extend(byt_conv(params[i]))
    buf2 = array2bytearray_little(kernel, 2)
    buf3 = array2bytearray_little(data_matrice, 2)
    seri.write(buf1)
    seri.write(buf2)
    seri.write(buf3)
    result = recv_res(seri, result_rows, result_cols)
    return result

def communicate_with_fpga(input_data, output_file, result_queue, mode):
    if mode == "Simülasyon":
        time.sleep(1)
        with open(output_file, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.float32)
        fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)
        result_queue.put((fpga_output, len(data)))
    else:
        try:
            ser = serial.Serial(
                port='COM1',
                baudrate=115200,
                stopbits=serial.STOPBITS_TWO,
                parity=serial.PARITY_NONE,
                timeout=2
            )

            # Çok kanallı giriş ve çoklu filtreler için işlem
            # input_data şekli: (batch_size, height, width, in_channels)
            # weights[0] şekli: (kernel_height, kernel_width, in_channels, out_channels)
            in_channels = input_data.shape[-1]  # Giriş kanalları
            out_channels = weights[0].shape[-1]  # Çıkış kanalları (filtre sayısı)
            data_rows, data_cols = input_data.shape[1:3]  # Yükseklik ve genişlik
            kernel_size = weights[0].shape[0]  # Kare kernel varsayımı

            # Çıktı boyutlarını hesapla
            result_rows = data_rows - kernel_size + 1
            result_cols = data_cols - kernel_size + 1
            # Toplam çıktı şekli: (batch_size, result_rows, result_cols, out_channels)
            fpga_output = np.zeros((input_data.shape[0], result_rows, result_cols, out_channels), dtype=np.float32)

            # Her giriş kanalı ve çıkış filtresi için döngü
            for out_ch in range(out_channels):  # Çıkış filtreleri
                for in_ch in range(in_channels):  # Giriş kanalları
                    # Giriş verisinden ilgili kanalı seç
                    data_matrice = input_data[0, :, :, in_ch]  # İlk batch, belirli kanal
                    # Ağırlıklardan ilgili filtreyi seç
                    kernel = weights[0][:, :, in_ch, out_ch]  # Belirli giriş/çıkış kanalı için filtre

                    # Conv_HW ile evrişim işlemini gerçekleştir
                    channel_output = Conv_HW(ser, data_rows, data_cols, kernel_size, data_matrice, kernel)

                    # Çıktıyı toplu sonuca ekle (kanallar arasında toplama)
                    fpga_output[0, :, :, out_ch] += channel_output  # İlk batch için

            # Bias ekleme (varsa)
            if len(weights) > 1:  # Bias varsa (weights[1])
                for out_ch in range(out_channels):
                    fpga_output[0, :, :, out_ch] += weights[1][out_ch]

            # Çıktıyı orijinal şekle uygun hale getir
            data_sent_size = (data_rows * data_cols * in_channels + kernel_size * kernel_size * in_channels * out_channels) * 4  # Yaklaşık veri boyutu

            ser.close()
            result_queue.put((fpga_output, data_sent_size))

        except Exception as e:
            available_ports = get_available_ports()
            error_msg = f"FPGA iletişimi hatası: {str(e)}\nMevcut portlar: {available_ports or 'Yok'}\nLütfen doğru portu kontrol edin veya FPGA'nın bağlı olduğundan emin olun."
            result_queue.put((None, 0, error_msg))

# Bölüm 6: Katman Çıktısı Hesaplama
# Bu bölümde, seçilen katmanın çıktısını hesaplayan bir fonksiyon tanımlanır.

def get_layer_output(model, layer_index, input_data):
    # Belirtilen katmanın çıktısını hesaplar
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Bölüm 7: Kağan için Veri Hazırlama
# Bu bölümde, ağırlıklar ve çıktılar binary formatta kaydedilir (FPGA için veri hazırlığı).

def save_data_for_kagan(weights, window_params, data_size, output_data):
    # Ağırlıkları, pencere parametrelerini ve çıktıları binary dosyaya kaydeder
    bin_file = f"data_for_kagan_{uuid.uuid4()}.bin"
    with open(bin_file, 'wb') as f:
        for w in weights:
            f.write(w.flatten().astype(np.float32).tobytes())
        if window_params:
            if "kernel_size" in window_params:
                f.write(struct.pack('ii', *window_params["kernel_size"]))
                f.write(struct.pack('ii', *window_params["stride"]))
                padding_val = 1 if window_params["padding"] == "valid" else 0
                f.write(struct.pack('i', padding_val))
            elif "units" in window_params:
                f.write(struct.pack('i', window_params["units"]))
        f.write(struct.pack('Q', data_size))
        f.write(output_data.flatten().astype(np.float32).tobytes())
    return bin_file

# Bölüm 8: Ana Simülasyon ve Çalıştırma Fonksiyonu
# Bu bölümde, modelin çalıştırılması, FPGA ile iletişim ve sonuçların karşılaştırılması gerçekleştirilir.

def run_model(model_type, model_selection, layer_selection, data_choice, mode):
    # Seçilen model, katman ve mod ile modeli çalıştırır ve FPGA çıktısını gerçek çıktı ile karşılaştırır
    model_info = MODEL_TYPES[model_type]["models"][model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]
    target_size = model_info["target_size"]
    file_format = MODEL_TYPES[model_type]["file_format"]

    # Modeli yükle
    try:
        loaded_model = tf.keras.models.load_model(model_path)
    except Exception as e:
        return f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Test verisini yükle
    try:
        if flow_from_directory:
            test_data = load_images_with_flow_from_directory(test_path, target_size=target_size, batch_size=8)
        else:
            test_data = load_images_from_directory(test_path, target_size=target_size, batch_size=8)
    except Exception as e:
        return f"Test verisi yükleme hatası: {str(e)}\nTest yolu: {test_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Katman seçimini işle (sadece Donanım modunda)
    layer_index = None
    current_layer = None
    if mode == "Donanım" and layer_selection and layer_selection != "None":
        try:
            layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
            current_layer = loaded_model.layers[layer_index]
        except (ValueError, IndexError) as e:
            return f"Katman seçimi hatası: {str(e)}\nSeçilen katman: {layer_selection}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Veri seçimini yap
    try:
        if data_choice == "Test Verisinden Örnek Kullan":
            if flow_from_directory:
                input_data = next(test_data)
            else:
                input_data = next(iter(test_data))
            data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
        else:
            expected_shape = loaded_model.input.shape[1:]
            input_data = np.random.random((8, *expected_shape))
            data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"
    except Exception as e:
        return f"Veri seçimi hatası: {str(e)}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    try:
        if mode == "Simülasyon" or layer_selection == "None" or not layer_selection:
            # Simülasyon modunda veya katman seçilmediğinde tam modeli çalıştır
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            bin_file = None
        else:
            # Donanım modunda seçilen katmanın çıktısını hesapla ve FPGA ile iletişim kur
            global weights  # Conv_HW için weights'i global yap (entegre ederken erişim için)
            weights, window_params = extract_weights_and_window(current_layer)
            real_output = get_layer_output(loaded_model, layer_index, input_data)

            output_file_x = f"file_{uuid.uuid4()}.x"
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())  # input_data yerine real_output (katman inputu) kullan; ama Conv_HW için data_matrice katman inputu olmalı.

            bin_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()), real_output)

            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=communicate_with_fpga,
                args=(real_output, output_file_x, result_queue, mode)  # input_data yerine katman inputu (real_output öncesi output)
            )
            comm_thread.start()
            comm_thread.join()

            fpga_result = result_queue.get()
            if len(fpga_result) == 3:
                raise Exception(fpga_result[2])
            fpga_output, data_sent_size = fpga_result
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            os.remove(output_file_x)
            if bin_file:
                os.remove(bin_file)

        # Çıktıları karşılaştır ve benzerlik skorunu hesapla
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        # Sonuçları formatla
        output_str = f"Seçilen Model Türü: {model_type}\n"
        output_str += f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        # Benzerlik skorunu görselleştir
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Bölüm 9: Gradio Arayüz Yardımcı Fonksiyonları
# Bu bölümde, Gradio arayüzünün dinamik bileşenlerini güncelleyen fonksiyonlar tanımlanır.

def update_model_dropdown(model_type):
    # Model türü seçildiğinde model seçim dropdown'unu günceller
    try:
        models = list(MODEL_TYPES[model_type]["models"].keys())
        return gr.Dropdown(choices=models, value=models[0], label="Model Seçimi")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Model Seçimi", info=f"Hata: {str(e)}")

def update_layer_dropdown(model_type, model_selection):
    # Model veya model türü değiştiğinde katman seçim dropdown'unu günceller
    try:
        model_info = MODEL_TYPES[model_type]["models"][model_selection]
        model_path = model_info["model_path"]
        if not os.path.exists(model_path):
            return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                               info=f"Model dosyası bulunamadı: {model_path}")

        loaded_model = tf.keras.models.load_model(model_path)
        layer_names = ["None"]
        for i, layer in enumerate(loaded_model.layers):
            try:
                output_shape = layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: {output_shape}")
            except Exception as e:
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: Hata ({str(e)})")
        return gr.Dropdown(choices=layer_names, value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                           info=f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}")

def update_layer_visibility(mode):
    # Çalışma modu değiştiğinde katman seçim dropdown'unun görünürlüğünü kontrol eder
    return gr.update(visible=(mode == "Donanım"))

# Bölüm 10: Gradio Kullanıcı Arayüzü
# Bu bölümde, interaktif Gradio arayüzü oluşturulur ve kullanıcı girişleri alınır.

with gr.Blocks(title="CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# CNN Model Hızlandırıcı")
    gr.Markdown("Model türünüzü ve modelinizi seçin, ardından katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")

    # Model türü seçimi
    model_type_input = gr.Dropdown(choices=list(MODEL_TYPES.keys()), label="Model Türü Seçimi", value="LeNet")

    # Model ve mod seçimi
    with gr.Row():
        model_input = gr.Dropdown(label="Model Seçimi")
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")

    # Katman seçimi ve simülasyon
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin", visible=False)
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Model türü değiştiğinde model dropdown'unu güncelle
    model_type_input.change(fn=update_model_dropdown, inputs=model_type_input, outputs=model_input)

    # Model veya model türü değiştiğinde katman dropdown'unu güncelle
    model_input.change(fn=update_layer_dropdown, inputs=[model_type_input, model_input], outputs=layer_input)
    model_type_input.change(fn=update_layer_dropdown, inputs=[model_type_input, model_input], outputs=layer_input)

    # Çalışma modu değiştiğinde katman dropdown'unun görünürlüğünü güncelle
    mode_input.change(fn=update_layer_visibility, inputs=mode_input, outputs=layer_input)

    # Modeli çalıştır
    submit_btn.click(fn=run_model, inputs=[model_type_input, model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Arayüzü başlat
interface.launch()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6f5036ede4692634dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#yeniyeni

# 26.08.2025 Grubun ortak Python kodu deneme1

# Kod Açıklaması Part Part

# Bölüm 1: Kütüphane İçe Aktarmaları ve Kurulum
# Bu bölümde, projede kullanılan tüm kütüphaneler içe aktarılır ve Google Drive bağlantısı kurulur.
# Gerekli kütüphaneler: TensorFlow (sinir ağı), NumPy (sayısal işlemler), Gradio (kullanıcı arayüzü), PySerial (FPGA iletişimi) vb.

!pip install pyserial
!pip install gradio==3.50.2
import tensorflow as tf  # Sinir ağı modellerini oluşturmak ve çalıştırmak için
import numpy as np  # Sayısal işlemler ve dizi manipülasyonları için
from tensorflow.keras import layers  # Sinir ağı katmanlarını tanımlamak için
import gradio as gr  # İnteraktif kullanıcı arayüzü oluşturmak için
import os  # Dosya ve dizin işlemleri için
import threading  # FPGA ile paralel iletişim için çoklu iş parçacığı
import queue  # İş parçacıkları arasında veri aktarımı için kuyruk
import time  # Simülasyon modunda gecikme simülasyonu için
import serial  # FPGA ile seri port üzerinden iletişim için
import struct  # Binary veri paketleme ve açma için
import serial.tools.list_ports  # Mevcut seri portları listelemek için
import platform  # İşletim sistemi bilgisini almak için
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Resim yükleme ve ön işleme için
import uuid  # Benzersiz dosya adları oluşturmak için
from google.colab import drive  # Google Drive bağlantısı için

# Google Drive'ı bağla: Modeller ve veri setleri Google Drive'da saklanıyor
drive.mount('/content/drive')

# Bölüm 2: Model ve Veri Yolu Tanımlamaları
# Bu bölümde, LeNet, MobileNet ve VGG-16 modellerinin yolları ve veri seti bilgileri tanımlanır.
# Her model türü için farklı varyantlar ve test veri yolları belirtilir.

MODEL_TYPES = {
    "LeNet": {
        "models": {
            "LeNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model1/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (32, 32)
            },
            "LeNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Lenets/Model2/lenet_model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (32, 32)
            },
            "LeNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/models/model.h5",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (32, 32)
            }
        },
        "file_format": "h5"
    },
    "MobileNet": {
        "models": {
            "MobilNet1": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet1/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "MobilNet2": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet2/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "MobilNet3": {
                "model_path": "/content/drive/MyDrive/Datasetsets/MobilNet3/saved_model/mobilenet_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    },
    "VGG-16": {
        "models": {
            "VGG16-1": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-lenetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Lenets/Lenetsetleri/test",
                "flow_from_directory": True,
                "target_size": (224, 224)
            },
            "VGG16-2": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-vgg16sets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/VGG-16/test",
                "flow_from_directory": False,
                "target_size": (224, 224)
            },
            "VGG16-3": {
                "model_path": "/content/drive/MyDrive/Modeller/vgg16model-mobilnetsets/saved_model/vgg16_model.keras",
                "test_path": "/content/drive/MyDrive/Datasetsets/Mobilnets/seg_pred",
                "flow_from_directory": False,
                "target_size": (224, 224)
            }
        },
        "file_format": "keras"
    }
}

# Bölüm 3: Veri Yükleme Fonksiyonları
# Bu bölümde, test verilerini yüklemek için iki fonksiyon tanımlanır:
# - load_images_from_directory: Alt klasörsüz dizinlerden resimleri yükler
# - load_images_with_flow_from_directory: Alt klasörlü dizinlerden resimleri yükler

def load_images_from_directory(directory, target_size, batch_size=8):
    # Alt klasörsüz dizinlerden resimleri yükler ve ön işleme yapar
    images = []
    if not os.path.exists(directory):
        raise ValueError(f"{directory} klasörü bulunamadı!")
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path) and filename.endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0  # Normalizasyon
            images.append(img_array)
    if not images:
        raise ValueError(f"{directory} dizininde geçerli resim dosyası bulunamadı!")
    images = np.array(images)
    return tf.data.Dataset.from_tensor_slices(images).batch(batch_size)

def load_images_with_flow_from_directory(directory, target_size, batch_size=8):
    # Alt klasörlü dizinlerden resimleri yükler ve veri artırma uygular
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    return test_data

# Bölüm 4: Ağırlık ve Pencere Parametreleri Çıkarma
# Bu bölümde, seçilen katmanın ağırlıklarını ve pencere parametrelerini (ör. kernel boyutu, adım) çıkaran bir fonksiyon tanımlanır.

def extract_weights_and_window(layer):
    # Katmanın ağırlıklarını ve pencere parametrelerini (Conv2D veya Dense katmanları için) çıkarır
    weights = layer.get_weights()
    window_params = {}
    if isinstance(layer, tf.keras.layers.Conv2D):
        window_params["kernel_size"] = list(layer.kernel_size)
        window_params["stride"] = list(layer.strides)
        window_params["padding"] = layer.padding
    elif isinstance(layer, tf.keras.layers.Dense):
        window_params["units"] = layer.units
    return weights, window_params

# Bölüm 5: Seri Port ve FPGA İletişimi
# Bu bölümde, seri portları listelemek ve FPGA ile iletişim kurmak için fonksiyonlar tanımlanır.

def get_available_ports():
    # Mevcut seri portları listeler
    ports = serial.tools.list_ports.comports()
    return [port.device for port in ports]

# FPGA kodundan entegre edilen fonksiyonlar
def float2hex(data: float) -> int:
    ret = int()
    ret = struct.unpack('!I', struct.pack('!f', data))[0]
    return ret

def to_four_bytes_little(data) -> bytearray:
    to4 = struct.Struct('<I').pack
    dummy = float2hex(data)
    ret = bytearray(to4(dummy & 0xFFFFFFFF))
    return ret

def array2bytearray_little(data: np.ndarray, dim: int) -> bytearray:
    ret = bytearray()
    if dim == 1:
        for i in range(0, data.shape[0]):
            ret.extend(to_four_bytes_little(data[i]))
        return ret
    elif dim == 2:
        for i in range(0, data.shape[0]):
            for j in range(0, data.shape[1]):
                ret.extend(to_four_bytes_little(data[i,j]))
        return ret
    else:
        return ret

def byt_conv(data: int) -> bytes:
    ret = data.to_bytes(4,'little')
    return ret

def intarray2bytearray(data: np.ndarray) -> bytearray:
    ret = bytearray()
    for i in range(0, len(data)):
        ret.extend(byt_conv(data[i]))
    return ret

def read_float(seri: serial) -> float:
    dat = bytes()
    dat = seri.read(4)
    dum = struct.unpack('<I', dat)[0]
    dat = struct.pack('>I', dum)
    datflo = struct.unpack('>f', dat)
    return datflo[0]

def recv_res(seri: serial, row, col):
    res = np.zeros((row,col), dtype=float)
    for i in range(0,row):
        for j in range(0,col):
            res[i,j] = read_float(seri)
    return res

def Conv_HW(seri: serial, data_rows: int, data_cols: int, kernel_size: int, data_matrice, kernel):
    result_rows = data_rows - kernel_size + 1
    result_cols = data_cols - kernel_size + 1
    ram1_size: int = kernel_size * kernel_size * 4
    ram2_size: int = data_rows * data_cols * 4
    ram3_size: int = result_cols * result_cols * 4
    params = [data_rows, data_cols, kernel_size, result_rows, ram1_size, ram2_size, ram3_size, result_cols]
    buf1 = bytearray()
    for i in range(0, len(params)):
        buf1.extend(byt_conv(params[i]))
    buf2 = array2bytearray_little(kernel, 2)
    buf3 = array2bytearray_little(data_matrice, 2)
    seri.write(buf1)
    seri.write(buf2)
    seri.write(buf3)
    result = recv_res(seri, result_rows, result_cols)
    return result

def communicate_with_fpga(input_data, output_file, result_queue, mode):
    if mode == "Simülasyon":
        time.sleep(1)
        with open(output_file, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.float32)
        fpga_output = data.reshape(input_data.shape) + np.random.normal(0, 0.05, input_data.shape)
        result_queue.put((fpga_output, len(data)))
    else:
        try:
            ser = serial.Serial(
                port='COM1',
                baudrate=115200,
                stopbits=serial.STOPBITS_TWO,
                parity=serial.PARITY_NONE,
                timeout=2
            )

            # Çok kanallı giriş ve çoklu filtreler için işlem
            # input_data şekli: (batch_size, height, width, in_channels)
            # weights[0] şekli: (kernel_height, kernel_width, in_channels, out_channels)
            in_channels = input_data.shape[-1]  # Giriş kanalları
            out_channels = weights[0].shape[-1]  # Çıkış kanalları (filtre sayısı)
            data_rows, data_cols = input_data.shape[1:3]  # Yükseklik ve genişlik
            kernel_size = weights[0].shape[0]  # Kare kernel varsayımı

            # Çıktı boyutlarını hesapla
            result_rows = data_rows - kernel_size + 1
            result_cols = data_cols - kernel_size + 1
            # Toplam çıktı şekli: (batch_size, result_rows, result_cols, out_channels)
            fpga_output = np.zeros((input_data.shape[0], result_rows, result_cols, out_channels), dtype=np.float32)

            # Her giriş kanalı ve çıkış filtresi için döngü
            for out_ch in range(out_channels):  # Çıkış filtreleri
                for in_ch in range(in_channels):  # Giriş kanalları
                    # Giriş verisinden ilgili kanalı seç
                    data_matrice = input_data[0, :, :, in_ch]  # İlk batch, belirli kanal
                    # Ağırlıklardan ilgili filtreyi seç
                    kernel = weights[0][:, :, in_ch, out_ch]  # Belirli giriş/çıkış kanalı için filtre

                    # Conv_HW ile evrişim işlemini gerçekleştir
                    channel_output = Conv_HW(ser, data_rows, data_cols, kernel_size, data_matrice, kernel)

                    # Çıktıyı toplu sonuca ekle (kanallar arasında toplama)
                    fpga_output[0, :, :, out_ch] += channel_output  # İlk batch için

            # Bias ekleme (varsa)
            if len(weights) > 1:  # Bias varsa (weights[1])
                for out_ch in range(out_channels):
                    fpga_output[0, :, :, out_ch] += weights[1][out_ch]

            # Çıktıyı orijinal şekle uygun hale getir
            data_sent_size = (data_rows * data_cols * in_channels + kernel_size * kernel_size * in_channels * out_channels) * 4  # Yaklaşık veri boyutu

            ser.close()
            result_queue.put((fpga_output, data_sent_size))

        except Exception as e:
            available_ports = get_available_ports()
            error_msg = f"FPGA iletişimi hatası: {str(e)}\nMevcut portlar: {available_ports or 'Yok'}\nLütfen doğru portu kontrol edin veya FPGA'nın bağlı olduğundan emin olun."
            result_queue.put((None, 0, error_msg))

# Bölüm 6: Katman Çıktısı Hesaplama
# Bu bölümde, seçilen katmanın çıktısını hesaplayan bir fonksiyon tanımlanır.

def get_layer_output(model, layer_index, input_data):
    # Belirtilen katmanın çıktısını hesaplar
    if layer_index == 0:
        intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
        return intermediate_model.predict(input_data, verbose=0)
    else:
        prev_output = input_data
        for i in range(layer_index):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[i].output)
            prev_output = intermediate_model.predict(input_data, verbose=0)
        intermediate_model = tf.keras.Sequential()
        for i in range(layer_index + 1):
            intermediate_model.add(model.layers[i])
        return intermediate_model.predict(input_data, verbose=0)

# Bölüm 7: Kağan için Veri Hazırlama
# Bu bölümde, ağırlıklar ve çıktılar binary formatta kaydedilir (FPGA için veri hazırlığı).

def save_data_for_kagan(weights, window_params, data_size, output_data):
    # Ağırlıkları, pencere parametrelerini ve çıktıları binary dosyaya kaydeder
    bin_file = f"data_for_kagan_{uuid.uuid4()}.bin"
    with open(bin_file, 'wb') as f:
        for w in weights:
            f.write(w.flatten().astype(np.float32).tobytes())
        if window_params:
            if "kernel_size" in window_params:
                f.write(struct.pack('ii', *window_params["kernel_size"]))
                f.write(struct.pack('ii', *window_params["stride"]))
                padding_val = 1 if window_params["padding"] == "valid" else 0
                f.write(struct.pack('i', padding_val))
            elif "units" in window_params:
                f.write(struct.pack('i', window_params["units"]))
        f.write(struct.pack('Q', data_size))
        f.write(output_data.flatten().astype(np.float32).tobytes())
    return bin_file

# Bölüm 8: Ana Simülasyon ve Çalıştırma Fonksiyonu
# Bu bölümde, modelin çalıştırılması, FPGA ile iletişim ve sonuçların karşılaştırılması gerçekleştirilir.

def run_model(model_type, model_selection, layer_selection, data_choice, mode):
    # Seçilen model, katman ve mod ile modeli çalıştırır ve FPGA çıktısını gerçek çıktı ile karşılaştırır
    model_info = MODEL_TYPES[model_type]["models"][model_selection]
    model_path = model_info["model_path"]
    test_path = model_info["test_path"]
    flow_from_directory = model_info["flow_from_directory"]
    target_size = model_info["target_size"]
    file_format = MODEL_TYPES[model_type]["file_format"]

    # Modeli yükle
    try:
        loaded_model = tf.keras.models.load_model(model_path)
    except Exception as e:
        return f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Test verisini yükle
    try:
        if flow_from_directory:
            test_data = load_images_with_flow_from_directory(test_path, target_size=target_size, batch_size=8)
        else:
            test_data = load_images_from_directory(test_path, target_size=target_size, batch_size=8)
    except Exception as e:
        return f"Test verisi yükleme hatası: {str(e)}\nTest yolu: {test_path}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Katman seçimini işle (sadece Donanım modunda)
    layer_index = None
    current_layer = None
    if mode == "Donanım" and layer_selection and layer_selection != "None":
        try:
            layer_index = int(layer_selection.split(":")[0].replace("Katman ", ""))
            current_layer = loaded_model.layers[layer_index]
        except (ValueError, IndexError) as e:
            return f"Katman seçimi hatası: {str(e)}\nSeçilen katman: {layer_selection}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    # Veri seçimini yap
    try:
        if data_choice == "Test Verisinden Örnek Kullan":
            if flow_from_directory:
                input_data = next(test_data)
            else:
                input_data = next(iter(test_data))
            data_info = f"Örnek veri kullanıldı. Şekil: {input_data.shape}"
        else:
            expected_shape = loaded_model.input.shape[1:]
            input_data = np.random.random((8, *expected_shape))
            data_info = f"Rastgele veri oluşturuldu. Şekil: {input_data.shape}"
    except Exception as e:
        return f"Veri seçimi hatası: {str(e)}", "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

    try:
        if mode == "Simülasyon" or layer_selection == "None" or not layer_selection:
            # Simülasyon modunda veya katman seçilmediğinde tam modeli çalıştır
            real_output = loaded_model.predict(input_data, verbose=0)
            fpga_output = real_output
            serialized_output = real_output.flatten().tobytes()
            data_sent_size = len(serialized_output)
            weights_info = "Hızlandırma yapılmadı, ağırlıklar kullanılmadı."
            bin_file = None
        else:
            # Donanım modunda seçilen katmanın çıktısını hesapla ve FPGA ile iletişim kur
            global weights  # Conv_HW için weights'i global yap (entegre ederken erişim için)
            weights, window_params = extract_weights_and_window(current_layer)
            real_output = get_layer_output(loaded_model, layer_index, input_data)

            output_file_x = f"file_{uuid.uuid4()}.x"
            with open(output_file_x, 'wb') as f:
                f.write(real_output.flatten().tobytes())  # input_data yerine real_output (katman inputu) kullan; ama Conv_HW için data_matrice katman inputu olmalı.

            bin_file = save_data_for_kagan(weights, window_params, len(real_output.flatten().tobytes()), real_output)

            result_queue = queue.Queue()
            comm_thread = threading.Thread(
                target=communicate_with_fpga,
                args=(real_output, output_file_x, result_queue, mode)  # input_data yerine katman inputu (real_output öncesi output)
            )
            comm_thread.start()
            comm_thread.join()

            fpga_result = result_queue.get()
            if len(fpga_result) == 3:
                raise Exception(fpga_result[2])
            fpga_output, data_sent_size = fpga_result
            weights_info = f"Ağırlıklar ve pencere parametreleri: {window_params}, Ağırlık boyutu: {sum(w.size for w in weights)} eleman"

            os.remove(output_file_x)
            if bin_file:
                os.remove(bin_file)

        # Çıktıları karşılaştır ve benzerlik skorunu hesapla
        real_output_np = real_output if isinstance(real_output, np.ndarray) else real_output.numpy()
        fpga_output_np = fpga_output if isinstance(fpga_output, np.ndarray) else fpga_output.numpy()
        serialized_output = real_output_np.flatten().tobytes()

        mse = np.mean((real_output_np - fpga_output_np) ** 2)
        similarity = max(0, 1 - mse)

        expected_input_shape = (loaded_model.input.shape[1:] if layer_index == 0 else
                               loaded_model.layers[layer_index - 1].output.shape[1:] if layer_index else
                               "Tam model")

        # Sonuçları formatla
        output_str = f"Seçilen Model Türü: {model_type}\n"
        output_str += f"Seçilen Model: {model_selection}\n"
        output_str += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok (Tam Model)'}\n"
        output_str += f"Mod: {mode}\n"
        output_str += f"Beklenen Giriş Şekli: {expected_input_shape}\n"
        output_str += f"Gerçek Giriş Şekli: {input_data.shape if layer_index == 0 else loaded_model.layers[layer_index - 1].output.shape if layer_index else input_data.shape}\n"
        output_str += f"{data_info}\n"
        output_str += f"Gerçek Çıktı Şekli: {real_output.shape}\n"
        output_str += f"FPGA Çıktı Şekli: {fpga_output.shape}\n"
        output_str += f"Kağan'a Gönderilen Veri Boyutu: {data_sent_size} bayt\n"
        output_str += f"{weights_info}\n"
        output_str += f"Gerçek Çıktı Örnek Veri: {real_output_np.flatten()[:5]}\n"
        output_str += f"FPGA Çıktı Örnek Veri: {fpga_output_np.flatten()[:5]}\n"
        output_str += f"Benzerlik Skoru: {similarity:.4f} (1’e yakınsa daha uyumlu)\n"

        # Benzerlik skorunu görselleştir
        animation_html = f"""
        <div style='width: 100%; background: #f0f0f0; border-radius: 5px;'>
            <div style='width: {similarity * 100}%; background: #4CAF50; height: 20px; border-radius: 5px;
                        transition: width 1s ease-in-out; text-align: center; color: white;'>
                {similarity * 100:.1f}%
            </div>
        </div>
        <p style='text-align: center;'>FPGA Çıktısının Gerçek Çıktıyla Uyumu</p>
        """

        return output_str, animation_html

    except Exception as e:
        error_msg = f"Hata oluştu: {str(e)}\n"
        error_msg += f"Seçilen Katman: {current_layer.name if current_layer else 'Hızlandırma Yok'}\n"
        error_msg += f"Giriş Verisi Şekli: {input_data.shape}\n"
        error_msg += f"Geçerli Katman Çıkış Şekli: {real_output.shape if 'real_output' in locals() else 'Hesaplanamadı'}\n"
        return error_msg, "<p>Hata nedeniyle animasyon oluşturulamadı.</p>"

# Bölüm 9: Gradio Arayüz Yardımcı Fonksiyonları
# Bu bölümde, Gradio arayüzünün dinamik bileşenlerini güncelleyen fonksiyonlar tanımlanır.

def update_model_dropdown(model_type):
    # Model türü seçildiğinde model seçim dropdown'unu günceller
    try:
        models = list(MODEL_TYPES[model_type]["models"].keys())
        return gr.Dropdown(choices=models, value=models[0], label="Model Seçimi")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Model Seçimi", info=f"Hata: {str(e)}")

def update_layer_dropdown(model_type, model_selection):
    # Model veya model türü değiştiğinde katman seçim dropdown'unu günceller
    try:
        if model_type not in MODEL_TYPES or model_selection not in MODEL_TYPES[model_type]["models"]:
            return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                               info=f"Geçersiz model türü ({model_type}) veya model seçimi ({model_selection})")

        model_info = MODEL_TYPES[model_type]["models"][model_selection]
        model_path = model_info["model_path"]
        if not os.path.exists(model_path):
            return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                               info=f"Model dosyası bulunamadı: {model_path}")

        loaded_model = tf.keras.models.load_model(model_path)
        layer_names = ["None"]
        for i, layer in enumerate(loaded_model.layers):
            try:
                output_shape = layer.output.shape if hasattr(layer, 'output') else 'Giriş Katmanı'
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: {output_shape}")
            except Exception as e:
                layer_names.append(f"Katman {i}: {layer.name} - Çıkış Şekli: Hata ({str(e)})")
        return gr.Dropdown(choices=layer_names, value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin")
    except Exception as e:
        return gr.Dropdown(choices=["None"], value="None", label="Hızlandırmak İstediğiniz Katmanı Seçin",
                           info=f"Model yükleme hatası: {str(e)}\nModel yolu: {model_path}")

def update_layer_visibility(mode):
    # Çalışma modu değiştiğinde katman seçim dropdown'unun görünürlüğünü kontrol eder
    return gr.update(visible=(mode == "Donanım"))

# Bölüm 10: Gradio Kullanıcı Arayüzü
# Bu bölümde, interaktif Gradio arayüzü oluşturulur ve kullanıcı girişleri alınır.

with gr.Blocks(title="CNN Model Hızlandırıcı") as interface:
    gr.Markdown("# CNN Model Hızlandırıcı")
    gr.Markdown("Model türünüzü ve modelinizi seçin, ardından katman ve giriş verisi tipini belirleyin. Simülasyon veya donanım modunda FPGA çıktısının gerçek çıktı ile uyumunu görün!")

    # Model türü seçimi
    model_type_input = gr.Dropdown(choices=list(MODEL_TYPES.keys()), label="Model Türü Seçimi", value="LeNet")

    # Model ve mod seçimi
    with gr.Row():
        model_input = gr.Dropdown(label="Model Seçimi", value="LeNet1")
        mode_input = gr.Radio(choices=["Simülasyon", "Donanım"], label="Çalışma Modu", value="Simülasyon")

    # Katman seçimi ve simülasyon
    with gr.Row():
        with gr.Column():
            layer_input = gr.Dropdown(label="Hızlandırmak İstediğiniz Katmanı Seçin", visible=False)
            data_input = gr.Radio(choices=["Test Verisinden Örnek Kullan", "Rastgele Veri Kullan"],
                                 label="Giriş Verisi Seçimi", value="Test Verisinden Örnek Kullan")
            submit_btn = gr.Button("Modeli Çalıştır", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Sonuçlar", lines=15)
            output_animation = gr.HTML(label="FPGA Çıktı Uyumu")

    # Model türü değiştiğinde model dropdown'unu güncelle
    model_type_input.change(fn=update_model_dropdown, inputs=model_type_input, outputs=model_input)

    # Model değiştiğinde katman dropdown'unu güncelle
    model_input.change(fn=update_layer_dropdown, inputs=[model_type_input, model_input], outputs=layer_input)

    # Çalışma modu değiştiğinde katman dropdown'unun görünürlüğünü güncelle
    mode_input.change(fn=update_layer_visibility, inputs=mode_input, outputs=layer_input)

    # Modeli çalıştır
    submit_btn.click(fn=run_model, inputs=[model_type_input, model_input, layer_input, data_input, mode_input], outputs=[output_text, output_animation])

# Arayüzü başlat
interface.launch()

/usr/local/lib/python3.12/dist-packages/gradio/components/dropdown.py:231: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: LeNet1 or set allow_custom_value=True.
  warnings.warn(


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://453ba769dae372624a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
